In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import tensorflow as tf
import numpy as np


#  Import and read the data.csv.
import pandas as pd 
data_df = pd.read_csv("archive/data.csv")
#data_df.head()

In [2]:
#Drop unnecessary columns
data_df = data_df.drop(["Unnamed: 0","Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Team", "Games","Year","City", "Nat_Pop", "GDP"], 1)

#data_df = data_df.drop(["GDP_Per_Cap"], 1)
data_df = data_df[data_df.GDP_Per_Cap != '#DIV/0!']

#Rename Global_Pop% and Rural_Pop% to make it easier to call in future
data_df.rename(columns = {'Global_Pop%':'Global_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Rural_Pop%':'Rural_Pop_Perc'}, inplace = True)
data_df.rename(columns = {'Global_GDP%':'Global_GDP_Perc'}, inplace = True)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/939271320.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Unnamed: 0","Unnamed: 0.1", "ID", "Name", "Sex", "Age", "Height", "Weight","Event", "Team", "Games","Year","City", "Nat_Pop", "GDP"], 1)


In [3]:
#Filter out season data to just summer
data_df = data_df[data_df["Season"].str.contains("Winter") == False]

#Filter out hockey data since it is a winter sport
data_df = data_df[data_df["Sport"].str.contains("Hockey") == False]

In [4]:
#Turn Medal Values into Boolean
data_df.fillna(value = False,
          inplace = True)

#Drop Season since no longer needed after filtering
data_df = data_df.drop(["Season"],1)
data_df.head()

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/516755359.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Season"],1)


,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375
50,NOR,Handball,Gold,0,0.000705,21.474,80.592683,97101.38727,0.007291


In [5]:
#Replace medal values to true to finish boolean
data_df = data_df.replace({"Gold" : True})
data_df = data_df.replace({"Silver" : True})
data_df = data_df.replace({"Bronze" : True})

In [6]:
#Begin binning data to make it easier to get dummies
#Find min and max of data types to create bins
data_df['Global_Pop_Perc'] = data_df['Global_Pop_Perc'].astype(float, errors = 'raise')

min_value = data_df['Global_Pop_Perc'].min()
max_value = data_df['Global_Pop_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

names = ["0","1","2","3","4","5","6","7","8"]
print(bins)
print (names)


1.45e-06
0.224549844
[1.45000000e-06 2.49512716e-02 4.99010931e-02 7.48509147e-02
 9.98007362e-02 1.24750558e-01 1.49700379e-01 1.74650201e-01
 1.99600022e-01 2.24549844e-01]
['0', '1', '2', '3', '4', '5', '6', '7', '8']


In [7]:
data_df["Global_Pop_Bin"] = pd.cut(data_df["Global_Pop_Perc"], bins, labels=names)
data_df

,NOC,Sport,Medal,Host,Global_Pop_Perc,Rural_Pop_Perc,Life_Expect,GDP_Per_Cap,Global_GDP_Perc,Global_Pop_Bin
0,CHN,Basketball,False,0,0.213515,71.8,69.587,366.5330438,0.016811,8
1,CHN,Judo,False,0,0.190308,48.111,75.602,6337.478113,0.114286,7
26,FIN,Badminton,False,0,0.000846,17.817,77.465854,24342.1392,0.00375,0
29,FIN,Athletics,False,0,0.000846,17.817,77.465854,24342.1392,0.00375,0
50,NOR,Handball,True,0,0.000705,21.474,80.592683,97101.38727,0.007291,0
...,...,...,...,...,...,...,...,...,...,...
204020,RUS,Athletics,True,0,0.022383,26.582,65.470976,4102.25644,0.013493,0
204024,USA,Football,False,0,0.054676,26.377,71.156098,6098.258185,0.339523,2
204025,RUS,Rowing,False,0,0.023961,26.65,65.483659,1773.575078,0.007738,0
204026,RUS,Rowing,False,0,0.022383,26.582,65.470976,4102.25644,0.013493,0


In [8]:
data_df['GDP_Per_Cap'] = data_df['GDP_Per_Cap'].astype(float, errors = 'raise')

min_value = data_df['GDP_Per_Cap'].min()
max_value = data_df['GDP_Per_Cap'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

0.0
192989.1895


In [9]:
data_df["GDP_Per_Cap"] = pd.cut(data_df["GDP_Per_Cap"], bins, labels=names)

In [10]:
data_df['Rural_Pop_Perc'] = data_df['Rural_Pop_Perc'].astype(float, errors = 'raise')

min_value = data_df['Rural_Pop_Perc'].min()
max_value = data_df['Rural_Pop_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

0.0
96.305


In [11]:
data_df["Rural_Pop_Bin"] = pd.cut(data_df["Rural_Pop_Perc"], bins, labels=names)

In [12]:
data_df['Life_Expect'] = data_df['Life_Expect'].astype(float, errors = 'raise')

min_value = data_df['Life_Expect'].min()
max_value = data_df['Life_Expect'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

data_df["Life_Expect_Bin"] = pd.cut(data_df["Life_Expect"], bins, labels=names)

0.0
85.41707317


In [13]:
data_df.dtypes

NOC                  object
Sport                object
Medal                  bool
Host                  int64
Global_Pop_Perc     float64
Rural_Pop_Perc      float64
Life_Expect         float64
GDP_Per_Cap        category
Global_GDP_Perc      object
Global_Pop_Bin     category
Rural_Pop_Bin      category
Life_Expect_Bin    category
dtype: object

In [14]:
min_value = data_df['Global_GDP_Perc'].min()
max_value = data_df['Global_GDP_Perc'].max()
print(min_value)
print(max_value)

#Create bins with min and max
bins = np.linspace(min_value,max_value,10)

names = ["0","1","2","3","4","5","6","7","8"]

data_df["Global_GDP_Perc_Bin"] = pd.cut(data_df["Global_GDP_Perc"], bins, labels=names)

False
0.3963503649635037


In [15]:
data_df = data_df.drop(["Global_Pop_Perc", "Rural_Pop_Perc", "Life_Expect", "Global_GDP_Perc"], 1)
data_df.head()

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/4282354263.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["Global_Pop_Perc", "Rural_Pop_Perc", "Life_Expect", "Global_GDP_Perc"], 1)


,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
0,CHN,Basketball,False,0,0,8,6,7,0
1,CHN,Judo,False,0,0,7,4,7,2
26,FIN,Badminton,False,0,1,0,1,8,0
29,FIN,Athletics,False,0,1,0,1,8,0
50,NOR,Handball,True,0,4,0,2,8,0


Athletics Logistic Regression

In [16]:
# Creating on the basis of Athlethics
Athletics_df = data_df[data_df['Sport'] == "Athletics"]
Athletics_df = Athletics_df.dropna()
 
print(Athletics_df)

        NOC      Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
29      FIN  Athletics  False     0           1              0             1   
62      FRA  Athletics  False     0           1              0             1   
81      EST  Athletics  False     0           0              0             2   
82      EST  Athletics  False     0           0              0             2   
91      FRA  Athletics  False     0           0              0             2   
...     ...        ...    ...   ...         ...            ...           ...   
203977  KAZ  Athletics  False     0           0              0             4   
203978  KAZ  Athletics  False     0           0              0             4   
203979  KAZ  Athletics  False     0           0              0             4   
204019  RUS  Athletics   True     0           0              0             2   
204020  RUS  Athletics   True     0           0              0             2   

       Life_Expect_Bin Global_GDP_Perc_

In [17]:
Athletics_dummies = pd.get_dummies(Athletics_df)
Athletics_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
29,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
62,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
81,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split

# Convert categorical data to numeric and separate target feature for training data
x_train = Athletics_dummies.drop(columns=['Medal'])
y_train = Athletics_dummies['Medal']

# Convert categorical data to numeric and separate target feature for testing data
x_test = Athletics_dummies.drop(columns=['Medal'])
y_test = Athletics_dummies['Medal']

In [19]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9218859667230915

In [20]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Athletics_df)

array([0.45340004, 0.44138183, 0.38498074, ..., 0.2682194 , 0.73571669,
       0.73571669])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
29,FIN,Athletics,False,0,1,0,1,8,0
62,FRA,Athletics,False,0,1,0,1,8,0
81,EST,Athletics,False,0,0,0,2,7,0
82,EST,Athletics,False,0,0,0,2,7,0
91,FRA,Athletics,False,0,0,0,2,7,1
...,...,...,...,...,...,...,...,...,...
203977,KAZ,Athletics,False,0,0,0,4,7,0
203978,KAZ,Athletics,False,0,0,0,4,7,0
203979,KAZ,Athletics,False,0,0,0,4,7,0
204019,RUS,Athletics,True,0,0,0,2,6,0


In [21]:
Athletics_df['Predictions'] = y_pred.tolist()

In [22]:
Athletics_final = Athletics_df.drop_duplicates(subset=['NOC'], keep='last')
Athletics_final = Athletics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Athletics_final

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3032431419.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Athletics_final = Athletics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
3132,OMN,Athletics,0.056710
93515,PLW,Athletics,0.142405
124976,SOM,Athletics,0.039474
130609,AGO,Athletics,0.039474
130654,BDI,Athletics,0.394120
...,...,...,...
203957,DEU,Athletics,0.464116
203959,NLD,Athletics,0.186889
203968,USA,Athletics,0.858066
203979,KAZ,Athletics,0.268219


##Swimming Logistics Regression

In [23]:
# Creating on the basis of Athlethics
Swimming_df = data_df[data_df['Sport'] == "Swimming"]
Swimming_df = Swimming_df.dropna()
  
print(Swimming_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
64      NLD  Swimming  False     0           1              0             2   
65      NLD  Swimming  False     0           1              0             2   
66      NLD  Swimming  False     0           1              0             2   
67      NLD  Swimming  False     0           1              0             2   
68      NLD  Swimming  False     0           1              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203887  RUS  Swimming  False     0           0              0             2   
203888  RUS  Swimming  False     0           0              0             2   
203942  NLD  Swimming  False     0           1              0             2   
203943  NLD  Swimming  False     0           1              0             2   
203944  NLD  Swimming   True     0           1              0             1   

       Life_Expect_Bin Global_GDP_Perc_Bin  
64    

In [24]:
Swimming_dummies = pd.get_dummies(Swimming_df)
Swimming_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
64,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
65,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
66,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [25]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Swimming_dummies.drop(columns=['Medal'])
y_train = Swimming_dummies['Medal']

In [26]:
x_test = Swimming_dummies.drop(columns=['Medal'])
y_test = Swimming_dummies['Medal']

In [27]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9132554711418528

In [28]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Swimming_df)

array([0.57584815, 0.57584815, 0.57584815, ..., 0.57584815, 0.57584815,
       0.64262404])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
64,NLD,Swimming,False,0,1,0,2,8,0
65,NLD,Swimming,False,0,1,0,2,8,0
66,NLD,Swimming,False,0,1,0,2,8,0
67,NLD,Swimming,False,0,1,0,2,8,0
68,NLD,Swimming,False,0,1,0,2,8,0
...,...,...,...,...,...,...,...,...,...
203887,RUS,Swimming,False,0,0,0,2,7,0
203888,RUS,Swimming,False,0,0,0,2,7,0
203942,NLD,Swimming,False,0,1,0,2,8,0
203943,NLD,Swimming,False,0,1,0,2,8,0


In [29]:
Swimming_df['Predictions'] = y_pred.tolist()

In [30]:
Swimming_final = Swimming_df.drop_duplicates(subset=['NOC'], keep='last')
Swimming_final = Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2203429247.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Swimming_final = Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [31]:
A = pd.merge(Athletics_final, Swimming_final, how='outer', on=['NOC', 'NOC'])
A

,NOC,Sport_x,Predictions_x,Sport_y,Predictions_y
0,OMN,Athletics,0.056710,Swimming,0.035608
1,PLW,Athletics,0.142405,Swimming,0.035608
2,SOM,Athletics,0.039474,NaN,NaN
3,AGO,Athletics,0.039474,Swimming,0.006516
4,BDI,Athletics,0.394120,Swimming,0.007109
...,...,...,...,...,...
182,DEU,Athletics,0.464116,Swimming,0.635384
183,NLD,Athletics,0.186889,Swimming,0.642624
184,USA,Athletics,0.858066,Swimming,0.964731
185,KAZ,Athletics,0.268219,Swimming,0.073415


In [32]:
A.rename(columns = {'Predictions_x':'Athletics', 'Predictions_y':'Swimming'}, inplace = True)
A = A.drop(["Sport_x","Sport_y"], 1)
A

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2218725990.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  A = A.drop(["Sport_x","Sport_y"], 1)


,NOC,Athletics,Swimming
0,OMN,0.056710,0.035608
1,PLW,0.142405,0.035608
2,SOM,0.039474,NaN
3,AGO,0.039474,0.006516
4,BDI,0.394120,0.007109
...,...,...,...
182,DEU,0.464116,0.635384
183,NLD,0.186889,0.642624
184,USA,0.858066,0.964731
185,KAZ,0.268219,0.073415


Gymnastics Logistic Regression

In [33]:
Gymnastics_df = data_df[data_df['Sport'] == "Gymnastics"]
Gymnastics_df = Gymnastics_df.dropna()

In [34]:
Gymnastics_dummies = pd.get_dummies(Gymnastics_df)
Gymnastics_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGD,NOC_BGR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
85,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
86,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
87,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [35]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Gymnastics_dummies.drop(columns=['Medal'])
y_train = Gymnastics_dummies['Medal']

x_test = Gymnastics_dummies.drop(columns=['Medal'])
y_test = Gymnastics_dummies['Medal']

In [36]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9492940640655837

In [37]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Gymnastics_df)

array([0.23424944, 0.23424944, 0.23424944, ..., 0.19703449, 0.19703449,
       0.19703449])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
85,ESP,Gymnastics,False,0,1,0,1,8,0
86,ESP,Gymnastics,False,0,1,0,1,8,0
87,ESP,Gymnastics,False,0,1,0,1,8,0
88,ESP,Gymnastics,False,0,1,0,1,8,0
89,ESP,Gymnastics,False,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
203680,ITA,Gymnastics,False,0,0,0,3,7,0
203681,ITA,Gymnastics,False,0,0,0,3,7,0
203682,ITA,Gymnastics,False,0,0,0,3,7,0
203683,ITA,Gymnastics,False,0,0,0,3,7,0


In [38]:
Gymnastics_df['Predictions'] = y_pred.tolist()

In [39]:
Gymnastics_final = Gymnastics_df.drop_duplicates(subset=['NOC'], keep='last')
Gymnastics_final = Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2535500066.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Gymnastics_final = Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Cycling Logistic Regression

In [40]:
Cycling_df = data_df[data_df['Sport'] == "Cycling"]
Cycling_df = Cycling_df.dropna()

In [41]:
Cycling_dummies = pd.get_dummies(Cycling_df)
Cycling_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
131,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
132,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
173,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [42]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Cycling_dummies.drop(columns=['Medal'])
y_train = Cycling_dummies['Medal']

In [43]:
x_test = Cycling_dummies.drop(columns=['Medal'])
y_test = Cycling_dummies['Medal']

In [44]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9057317427989525

In [45]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Cycling_df)

array([0.54309791, 0.54309791, 0.04105731, ..., 0.58117185, 0.58117185,
       0.02388469])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
131,RUS,Cycling,True,0,0,0,2,6,0
132,RUS,Cycling,False,0,0,0,2,6,0
173,EGY,Cycling,False,0,0,0,5,7,0
204,USA,Cycling,False,0,2,1,1,8,5
241,EGY,Cycling,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203743,CHE,Cycling,False,0,2,0,2,8,0
203744,CHE,Cycling,False,0,2,0,2,8,0
203745,CHE,Cycling,False,0,1,0,2,8,0
203746,CHE,Cycling,False,0,1,0,2,8,0


In [46]:
Cycling_df['Predictions'] = y_pred.tolist()

In [47]:
Cycling_final = Cycling_df.drop_duplicates(subset=['NOC'], keep='last')
Cycling_final = Cycling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1199427753.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Cycling_final = Cycling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [48]:
B = pd.merge(Gymnastics_final, Cycling_final, how='outer', on=['NOC', 'NOC'])
B.rename(columns = {'Predictions_x':'Gymnastics', 'Predictions_y':'Cycling'}, inplace = True)
B = B.drop(["Sport_x","Sport_y"], 1)
B

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/597712745.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  B = B.drop(["Sport_x","Sport_y"], 1)


,NOC,Gymnastics,Cycling
0,YEM,0.018416,NaN
1,PAN,0.017619,NaN
2,TUN,0.021851,0.137522
3,BGD,0.021851,NaN
4,BRB,0.021851,0.003409
...,...,...,...
123,KHM,NaN,0.032306
124,HKG,NaN,0.063620
125,MWI,NaN,0.032306
126,LBN,NaN,0.104414


Rowing Logistic Regression

In [49]:
Rowing_df = data_df[data_df['Sport'] == "Rowing"]
Rowing_df = Rowing_df.dropna()

In [50]:
Rowing_dummies = pd.get_dummies(Rowing_df)
Rowing_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
60,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
61,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
98,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [51]:
# Convert categorical data to numeric and separate target feature for training data
x_train = Rowing_dummies.drop(columns=['Medal'])
y_train = Rowing_dummies['Medal']

x_test = Rowing_dummies.drop(columns=['Medal'])
y_test = Rowing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7811712846347607

In [52]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Rowing_df)

array([0.5895253 , 0.5895253 , 0.50789496, ..., 0.5895253 , 0.38279209,
       0.38279209])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
60,NLD,Rowing,True,0,1,0,2,8,0
61,NLD,Rowing,False,0,1,0,2,8,0
98,ITA,Rowing,True,0,1,0,2,8,0
116,ARG,Rowing,False,0,0,0,0,7,0
130,CHL,Rowing,False,0,0,0,1,7,0
...,...,...,...,...,...,...,...,...,...
203971,NLD,Rowing,False,0,0,0,2,8,0
203972,NLD,Rowing,True,0,1,0,2,8,0
203973,NLD,Rowing,True,0,1,0,2,8,0
204025,RUS,Rowing,False,0,0,0,2,6,0


In [53]:
Rowing_df['Predictions'] = y_pred.tolist()
Rowing_final = Rowing_df.drop_duplicates(subset=['NOC'], keep='last')
Rowing_final = Rowing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


C:\Users\14043\AppData\Local\Temp/ipykernel_1700/772740510.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Rowing_final = Rowing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Shooting Logistic Regression

In [54]:
Shooting_df = data_df[data_df['Sport'] == "Shooting"]
Shooting_df = Shooting_df.dropna()

In [55]:
Shooting_dummies = pd.get_dummies(Shooting_df)
Shooting_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
79,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
124,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
137,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [56]:
x_train = Shooting_dummies.drop(columns=['Medal'])
y_train = Shooting_dummies['Medal']

x_test = Shooting_dummies.drop(columns=['Medal'])
y_test = Shooting_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9352194180999839

In [57]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Shooting_df)

array([0.19086128, 0.28391255, 0.28391255, ..., 0.44241322, 0.19086128,
       0.19086128])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
79,NOR,Shooting,False,0,0,0,4,7,0
124,CHL,Shooting,False,0,0,0,1,7,0
137,CHL,Shooting,False,0,0,0,1,7,0
237,EGY,Shooting,False,0,0,0,5,7,0
282,EGY,Shooting,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203765,CHE,Shooting,False,0,2,0,2,8,0
203766,CHE,Shooting,False,0,2,0,2,8,0
203767,CHE,Shooting,False,0,2,0,2,8,0
203948,NOR,Shooting,False,0,0,0,4,7,0


In [58]:
Shooting_df['Predictions'] = y_pred.tolist()
Shooting_final = Shooting_df.drop_duplicates(subset=['NOC'], keep='last')
Shooting_final = Shooting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/4008684553.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Shooting_final = Shooting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [59]:
C = pd.merge(Rowing_final, Shooting_final, how='outer', on=['NOC', 'NOC'])
C.rename(columns = {'Predictions_x':'Rowing', 'Predictions_y':'Shooting'}, inplace = True)
C = C.drop(["Sport_x","Sport_y"], 1)
C

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2485172735.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  C = C.drop(["Sport_x","Sport_y"], 1)


,NOC,Rowing,Shooting
0,KWT,0.192253,0.534172
1,TGO,0.008707,NaN
2,HND,0.091612,0.190861
3,TTO,0.187417,0.145890
4,CMR,0.103862,NaN
...,...,...,...
125,DOM,NaN,0.109686
126,BHR,NaN,0.216840
127,MAR,NaN,0.000967
128,MDA,NaN,0.646543


Fencing Logistic Regression

In [60]:
Fencing_df = data_df[data_df['Sport'] == "Fencing"]
Fencing_df = Fencing_df.dropna()
Fencing_df

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
99,ESP,Fencing,True,0,1,0,2,8,0
117,GRC,Fencing,False,0,1,0,2,8,0
141,MEX,Fencing,False,1,0,0,3,6,0
142,MEX,Fencing,False,1,0,0,3,6,0
143,MEX,Fencing,False,1,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...
203312,AUT,Fencing,False,0,0,0,3,7,0
203313,AUT,Fencing,False,0,0,0,3,7,0
203408,CHN,Fencing,False,0,0,8,7,7,0
203409,CHN,Fencing,False,0,0,8,7,7,0


In [61]:
Fencing_dummies = pd.get_dummies(Fencing_df)
Fencing_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BEN,NOC_BFA,NOC_BGR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
99,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
117,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
141,False,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [62]:
x_train = Fencing_dummies.drop(columns=['Medal'])
y_train = Fencing_dummies['Medal']

x_test = Fencing_dummies.drop(columns=['Medal'])
y_test = Fencing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8693467336683417

In [63]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Fencing_df)

array([0.09263812, 0.04414394, 0.05364436, ..., 0.1206045 , 0.1206045 ,
       0.56245881])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
99,ESP,Fencing,True,0,1,0,2,8,0
117,GRC,Fencing,False,0,1,0,2,8,0
141,MEX,Fencing,False,1,0,0,3,6,0
142,MEX,Fencing,False,1,0,0,3,6,0
143,MEX,Fencing,False,1,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...
203312,AUT,Fencing,False,0,0,0,3,7,0
203313,AUT,Fencing,False,0,0,0,3,7,0
203408,CHN,Fencing,False,0,0,8,7,7,0
203409,CHN,Fencing,False,0,0,8,7,7,0


In [64]:
Fencing_df['Predictions'] = y_pred.tolist()
Fencing_final = Fencing_df.drop_duplicates(subset=['NOC'], keep='last')
Fencing_final = Fencing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2959214136.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Fencing_final = Fencing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Canoeing Logistic Regression

In [65]:
Canoeing_df = data_df[data_df['Sport'] == "Canoeing"]
Canoeing_df = Canoeing_df.dropna()

In [66]:
Canoeing_dummies = pd.get_dummies(Canoeing_df)
Canoeing_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
109,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
110,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
111,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [67]:
x_train = Canoeing_dummies.drop(columns=['Medal'])
y_train = Canoeing_dummies['Medal']

x_test = Canoeing_dummies.drop(columns=['Medal'])
y_test = Canoeing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8574193548387097

In [68]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Canoeing_df)

array([0.32177805, 0.32177805, 0.75111045, ..., 0.33041141, 0.33041141,
       0.39590712])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
109,CUB,Canoeing,False,0,0,0,2,7,0
110,CUB,Canoeing,False,0,0,0,2,7,0
111,BLR,Canoeing,False,0,0,0,2,7,0
112,BLR,Canoeing,True,0,0,0,2,7,0
203,CAN,Canoeing,False,1,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
203817,USA,Canoeing,False,0,1,1,1,8,6
203818,USA,Canoeing,False,0,1,1,1,8,6
203819,USA,Canoeing,False,0,2,1,1,8,5
203820,USA,Canoeing,False,0,2,1,1,8,5


In [69]:
Canoeing_df['Predictions'] = y_pred.tolist()
Canoeing_final = Canoeing_df.drop_duplicates(subset=['NOC'], keep='last')
Canoeing_final = Canoeing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2271154440.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Canoeing_final = Canoeing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [70]:
D = pd.merge(Fencing_final, Canoeing_final, how='outer', on=['NOC', 'NOC'])
D.rename(columns = {'Predictions_x':'Fencing', 'Predictions_y':'Canoeing'}, inplace = True)
D = D.drop(["Sport_x","Sport_y"], 1)
D

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/242490794.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  D = D.drop(["Sport_x","Sport_y"], 1)


,NOC,Fencing,Canoeing
0,QAT,0.033491,NaN
1,BEN,0.060600,NaN
2,PRY,0.060600,NaN
3,PER,0.043265,0.240127
4,SAU,0.060600,NaN
...,...,...,...
98,MKD,NaN,0.317249
99,SVK,NaN,0.746850
100,SRB,NaN,0.233500
101,SVN,NaN,0.280128


Wrestling Logistic Regression

In [71]:
Wrestling_df = data_df[data_df['Sport'] == "Wrestling"]
Wrestling_df = Wrestling_df.dropna()

In [72]:
Wrestling_dummies = pd.get_dummies(Wrestling_df)
Wrestling_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_ALB,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
52,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
53,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
84,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [73]:
x_train = Wrestling_dummies.drop(columns=['Medal'])
y_train = Wrestling_dummies['Medal']

x_test = Wrestling_dummies.drop(columns=['Medal'])
y_test = Wrestling_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.861604997597309

In [74]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Wrestling_df)

array([0.37197665, 0.37197665, 0.22172613, ..., 0.76725942, 0.18305438,
       0.49651937])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
52,NOR,Wrestling,False,0,1,0,2,8,0
53,NOR,Wrestling,False,0,2,0,2,8,0
84,EST,Wrestling,False,0,0,0,2,7,0
125,USA,Wrestling,True,0,1,1,1,8,6
207,EGY,Wrestling,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203710,HRV,Wrestling,False,0,0,0,3,8,0
203711,HRV,Wrestling,False,0,0,0,3,8,0
203779,USA,Wrestling,False,1,1,1,2,8,5
203924,LVA,Wrestling,False,0,0,0,3,7,0


In [75]:
Wrestling_df['Predictions'] = y_pred.tolist()
Wrestling_final = Wrestling_df.drop_duplicates(subset=['NOC'], keep='last')
Wrestling_final = Wrestling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1911261898.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Wrestling_final = Wrestling_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Football Logistic Regression

In [76]:
Football_df = data_df[data_df['Sport'] == "Football"]
Football_df = Football_df.dropna()
 
print(Football_df)

        NOC     Sport  Medal  Host GDP_Per_Cap Global_Pop_Bin Rural_Pop_Bin  \
69      NOR  Football   True     0           1              0             2   
118     CMR  Football   True     0           0              0             5   
122     GRC  Football  False     0           1              0             2   
134     ITA  Football  False     0           1              0             2   
166     IRQ  Football  False     0           0              0             2   
...     ...       ...    ...   ...         ...            ...           ...   
203675  SAU  Football  False     0           0              0             1   
203725  NLD  Football  False     0           2              0             1   
203784  ESP  Football  False     0           0              0             2   
203859  CAN  Football  False     0           2              0             1   
204024  USA  Football  False     0           0              2             2   

       Life_Expect_Bin Global_GDP_Perc_Bin  
69    

In [77]:
Football_dummies = pd.get_dummies(Football_df)
Football_dummies.head(3)

,Medal,Host,NOC_ARE,NOC_ARG,NOC_AUS,NOC_BEL,NOC_BLR,NOC_BRA,NOC_CAN,NOC_CHE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
69,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
118,True,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
122,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [78]:
x_train = Football_dummies.drop(columns=['Medal'])
y_train = Football_dummies['Medal']

x_test = Football_dummies.drop(columns=['Medal'])
y_test = Football_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8740793537657401

In [79]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Football_df)

array([0.89871808, 0.80488485, 0.04849552, ..., 0.29936204, 0.91046264,
       0.01814279])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
69,NOR,Football,True,0,1,0,2,8,0
118,CMR,Football,True,0,0,0,5,5,0
122,GRC,Football,False,0,1,0,2,8,0
134,ITA,Football,False,0,1,0,2,8,0
166,IRQ,Football,False,0,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
203675,SAU,Football,False,0,0,0,1,7,0
203725,NLD,Football,False,0,2,0,1,8,0
203784,ESP,Football,False,0,0,0,2,7,0
203859,CAN,Football,False,0,2,0,1,8,0


In [80]:
Football_df['Predictions'] = y_pred.tolist()
Football_final = Football_df.drop_duplicates(subset=['NOC'], keep='last')
Football_final = Football_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3633843560.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Football_final = Football_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [81]:
E = pd.merge(Wrestling_final, Football_final, how='outer', on=['NOC', 'NOC'])
E.rename(columns = {'Predictions_x':'Wrestling', 'Predictions_y':'Football'}, inplace = True)
E = E.drop(["Sport_x","Sport_y"], 1)
E

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3963166347.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  E = E.drop(["Sport_x","Sport_y"], 1)


,NOC,Wrestling,Football
0,YEM,0.024006,NaN
1,CHL,0.196641,0.892639
2,BHR,0.298727,NaN
3,LBN,0.165723,NaN
4,CIV,0.050638,0.000010
...,...,...,...
123,CRI,NaN,0.026713
124,GHA,NaN,0.139070
125,PRY,NaN,0.807502
126,KWT,NaN,0.022682


Boxing Logistic Regression

In [82]:
Boxing_df = data_df[data_df['Sport'] == "Boxing"]
Boxing_df = Boxing_df.dropna()


In [83]:
Boxing_dummies = pd.get_dummies(Boxing_df)
Boxing_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_AGO,NOC_ARG,NOC_ARM,NOC_ATG,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
80,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
94,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
102,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [84]:
x_train = Boxing_dummies.drop(columns=['Medal'])
y_train = Boxing_dummies['Medal']

x_test = Boxing_dummies.drop(columns=['Medal'])
y_test = Boxing_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8773800456968773

In [85]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Boxing_df)

array([0.29803402, 0.62643175, 0.04655201, ..., 0.64027763, 0.3415265 ,
       0.3415265 ])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
80,MAR,Boxing,False,0,0,0,3,7,0
94,BGR,Boxing,False,0,0,0,3,7,0
102,SDN,Boxing,False,0,0,0,7,5,0
119,CMR,Boxing,False,0,0,0,7,4,0
145,NGA,Boxing,False,0,0,0,7,4,0
...,...,...,...,...,...,...,...,...,...
203780,CHL,Boxing,False,0,0,0,2,6,0
203788,COL,Boxing,False,0,0,0,3,7,0
203857,ITA,Boxing,False,0,0,0,3,7,0
203877,BLR,Boxing,True,0,0,0,2,7,0


In [86]:
Boxing_df['Predictions'] = y_pred.tolist()
Boxing_final = Boxing_df.drop_duplicates(subset=['NOC'], keep='last')
Boxing_final = Boxing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/4182983223.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Boxing_final = Boxing_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Equestrianism Logistic Regression

In [87]:
Equestrianism_df = data_df[data_df['Sport'] == "Equestrianism"]
Equestrianism_df = Equestrianism_df.dropna()

In [88]:
Equestrianism_dummies = pd.get_dummies(Equestrianism_df)
Equestrianism_dummies.head(3)

,Medal,Host,NOC_ARE,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BLR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
73,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
196,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
197,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [89]:
x_train = Equestrianism_dummies.drop(columns=['Medal'])
y_train = Equestrianism_dummies['Medal']

x_test = Equestrianism_dummies.drop(columns=['Medal'])
y_test = Equestrianism_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8697627460878344

In [90]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Equestrianism_df)

array([0.0793335 , 0.05436557, 0.05436557, ..., 0.19635419, 0.19635419,
       0.19635419])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
73,NOR,Equestrianism,False,0,1,0,2,8,0
196,IRL,Equestrianism,False,0,2,0,3,8,0
197,IRL,Equestrianism,False,0,2,0,3,8,0
1037,ARG,Equestrianism,False,0,0,0,2,6,0
1038,ARG,Equestrianism,False,0,0,0,2,6,0
...,...,...,...,...,...,...,...,...,...
203407,ARG,Equestrianism,False,0,0,0,1,7,0
203598,DNK,Equestrianism,False,0,2,0,1,8,0
203599,DNK,Equestrianism,True,0,2,0,1,8,0
203600,DNK,Equestrianism,False,0,2,0,1,8,0


In [91]:
Equestrianism_df['Predictions'] = y_pred.tolist()
Equestrianism_final = Equestrianism_df.drop_duplicates(subset=['NOC'], keep='last')
Equestrianism_final = Equestrianism_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1249077138.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Equestrianism_final = Equestrianism_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [92]:
F = pd.merge(Boxing_final, Equestrianism_final, how='outer', on=['NOC', 'NOC'])
F.rename(columns = {'Predictions_x':'Boxing', 'Predictions_y':'Equestrianism'}, inplace = True)
F = F.drop(["Sport_x","Sport_y"], 1)
F

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1904098448.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  F = F.drop(["Sport_x","Sport_y"], 1)


,NOC,Boxing,Equestrianism
0,OMN,0.270712,NaN
1,KWT,0.217088,NaN
2,SLB,0.133535,NaN
3,BOL,0.133535,0.041644
4,FSM,0.094315,NaN
...,...,...,...
147,COL,0.432454,0.075769
148,ITA,0.640278,0.414043
149,BLR,0.341526,0.112318
150,ARE,NaN,0.124629


Judo Logistic Regression

In [93]:
Judo_df = data_df[data_df['Sport'] == "Judo"]
Judo_df = Judo_df.dropna()

In [94]:
Judo_dummies = pd.get_dummies(Judo_df)
Judo_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_AFG,NOC_AGO,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
1,False,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
100,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
120,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [95]:
x_train = Judo_dummies.drop(columns=['Medal'])
y_train = Judo_dummies['Medal']

x_test = Judo_dummies.drop(columns=['Medal'])
y_test = Judo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8726263511539585

In [96]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Judo_df)

array([0.64518045, 0.00319859, 0.29464617, ..., 0.57990675, 0.69661665,
       0.64376888])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
1,CHN,Judo,False,0,0,7,4,7,2
100,TCD,Judo,False,0,0,0,7,4,0
120,TUR,Judo,False,0,0,0,3,7,0
300,EGY,Judo,False,0,0,0,5,7,0
314,EGY,Judo,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203810,CAN,Judo,False,0,2,0,1,8,0
203811,CAN,Judo,False,0,1,0,1,8,0
203922,GEO,Judo,True,0,0,0,4,7,0
203960,NLD,Judo,True,0,1,0,2,8,0


In [97]:
Judo_df['Predictions'] = y_pred.tolist()
Judo_final = Judo_df.drop_duplicates(subset=['NOC'], keep='last')
Judo_final = Judo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/405281857.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Judo_final = Judo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Basketball Logistic Regression

In [98]:
Basketball_df = data_df[data_df['Sport'] == "Basketball"]
Basketball_df = Basketball_df.dropna()

In [99]:
Basketball_dummies = pd.get_dummies(Basketball_df)
Basketball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAF,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
0,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
107,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
192,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [100]:
x_train = Basketball_dummies.drop(columns=['Medal'])
y_train = Basketball_dummies['Medal']

x_test = Basketball_dummies.drop(columns=['Medal'])
y_test = Basketball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8938140919616275

In [101]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Basketball_df)

array([0.47147875, 0.84547076, 0.26249214, ..., 0.61633405, 0.00206276,
       0.01347241])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
0,CHN,Basketball,False,0,0,8,6,7,0
107,ESP,Basketball,False,0,1,0,2,8,0
192,ITA,Basketball,False,0,0,0,3,8,0
257,EGY,Basketball,False,0,0,0,5,5,0
264,EGY,Basketball,False,0,0,0,5,6,0
...,...,...,...,...,...,...,...,...,...
203733,LTU,Basketball,False,0,0,0,3,7,0
203734,LTU,Basketball,True,0,0,0,3,7,0
203735,LTU,Basketball,False,0,0,0,3,7,0
203956,DEU,Basketball,False,0,2,0,2,8,1


In [102]:
Basketball_df['Predictions'] = y_pred.tolist()
Basketball_final = Basketball_df.drop_duplicates(subset=['NOC'], keep='last')
Basketball_final = Basketball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1911532976.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Basketball_final = Basketball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [103]:
G = pd.merge(Judo_final, Basketball_final, how='outer', on=['NOC', 'NOC'])
G.rename(columns = {'Predictions_x':'Judo', 'Predictions_y':'Basketball'}, inplace = True)
G = G.drop(["Sport_x","Sport_y"], 1)
G

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3586753422.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  G = G.drop(["Sport_x","Sport_y"], 1)


,NOC,Judo,Basketball
0,GUM,0.133986,NaN
1,TGO,0.068382,NaN
2,NIC,0.180143,NaN
3,TTO,0.180143,NaN
4,BEN,0.068382,NaN
...,...,...,...
152,HUN,0.430885,NaN
153,CHL,0.174665,NaN
154,CAN,0.194549,0.002093
155,GEO,0.579907,NaN


Handball Logistic Regression

In [104]:
Handball_df = data_df[data_df['Sport'] == "Handball"]
Handball_df = Handball_df.dropna()

In [105]:
Handball_dummies = pd.get_dummies(Handball_df)
Handball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BRA,NOC_CAN,NOC_CHE,NOC_CHN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
50,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
113,True,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
114,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [106]:
x_train = Handball_dummies.drop(columns=['Medal'])
y_train = Handball_dummies['Medal']

x_test = Handball_dummies.drop(columns=['Medal'])
y_test = Handball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.7936344969199178

In [107]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Handball_df)

array([0.9872138 , 0.58509065, 0.75110245, ..., 0.51197043, 0.02492861,
       0.0236445 ])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
50,NOR,Handball,True,0,4,0,2,8,0
113,FRA,Handball,True,0,2,0,2,8,1
114,FRA,Handball,True,0,1,0,1,8,0
115,FRA,Handball,True,0,1,0,1,8,0
135,FRA,Handball,False,0,1,0,2,8,1
...,...,...,...,...,...,...,...,...,...
203758,CUB,Handball,False,0,0,0,2,7,0
203770,CHE,Handball,False,0,2,0,2,8,0
203787,ESP,Handball,False,0,0,0,2,8,0
203858,AUT,Handball,False,0,1,0,3,7,0


In [108]:
Handball_df['Predictions'] = y_pred.tolist()
Handball_final = Handball_df.drop_duplicates(subset=['NOC'], keep='last')
Handball_final = Handball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1399169446.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Handball_final = Handball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Volleyball

In [109]:
Volleyball_df = data_df[data_df['Sport'] == "Volleyball"]
Volleyball_df = Volleyball_df.dropna()

In [110]:
Volleyball_dummies = pd.get_dummies(Volleyball_df)
Volleyball_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,NOC_CHN,NOC_CMR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
210,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
218,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
236,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [111]:
x_train = Volleyball_dummies.drop(columns=['Medal'])
y_train = Volleyball_dummies['Medal']

x_test = Volleyball_dummies.drop(columns=['Medal'])
y_test = Volleyball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8166845685642679

In [112]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Volleyball_df)

array([0.03059131, 0.03059131, 0.03059131, ..., 0.42613874, 0.42613874,
       0.01264144])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
210,EGY,Volleyball,False,0,0,0,5,7,0
218,EGY,Volleyball,False,0,0,0,5,7,0
236,EGY,Volleyball,False,0,0,0,5,7,0
254,EGY,Volleyball,False,0,0,0,5,6,0
270,EGY,Volleyball,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203795,CHN,Volleyball,False,0,0,8,7,7,0
203945,NLD,Volleyball,False,0,0,0,2,8,0
203946,NLD,Volleyball,True,0,1,0,2,8,0
203947,NLD,Volleyball,True,0,1,0,2,8,0


In [113]:
Volleyball_df['Predictions'] = y_pred.tolist()
Volleyball_final = Volleyball_df.drop_duplicates(subset=['NOC'], keep='last')
Volleyball_final = Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3185448111.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Volleyball_final = Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [114]:
H = pd.merge(Handball_final, Volleyball_final, how='outer', on=['NOC', 'NOC'])
H.rename(columns = {'Predictions_x':'Handball', 'Predictions_y':'Volleyball'}, inplace = True)
H = H.drop(["Sport_x","Sport_y"], 1)
H

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1895146923.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  H = H.drop(["Sport_x","Sport_y"], 1)


,NOC,Handball,Volleyball
0,COG,0.057339,NaN
1,KWT,0.011314,NaN
2,DZA,0.007170,0.015409
3,AUS,0.051609,0.033935
4,ISL,0.478225,NaN
5,NOR,0.481155,NaN
6,CZE,0.011314,NaN
7,CAN,0.014495,0.018650
8,ARG,0.025113,0.450339
9,CIV,0.009617,NaN


Weightlifting

In [115]:
Weightlifting_df = data_df[data_df['Sport'] == "Weightlifting"]
Weightlifting_df = Weightlifting_df.dropna()

In [116]:
Weightlifting_dummies = pd.get_dummies(Weightlifting_df)
Weightlifting_dummies.head(3)

,Medal,Host,NOC_ABW,NOC_ALB,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
51,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
165,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
176,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [117]:
x_train = Weightlifting_dummies.drop(columns=['Medal'])
y_train = Weightlifting_dummies['Medal']

x_test = Weightlifting_dummies.drop(columns=['Medal'])
y_test = Weightlifting_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8793300342596118

In [118]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Weightlifting_df)

array([0.46818654, 0.1214001 , 0.44990507, ..., 0.24977166, 0.33853195,
       0.65612038])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
51,ROU,Weightlifting,False,0,0,0,4,7,0
165,IRQ,Weightlifting,False,0,0,0,4,5,0
176,QAT,Weightlifting,False,0,1,0,0,8,0
188,AZE,Weightlifting,False,0,0,0,4,7,0
195,IRQ,Weightlifting,False,0,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
203502,MAR,Weightlifting,False,0,0,0,4,7,0
203658,CZE,Weightlifting,False,0,0,0,2,7,0
203855,ESP,Weightlifting,False,0,0,0,2,8,0
203856,ESP,Weightlifting,False,1,0,0,2,8,0


In [119]:
Weightlifting_df['Predictions'] = y_pred.tolist()
Weightlifting_final = Weightlifting_df.drop_duplicates(subset=['NOC'], keep='last')
Weightlifting_final = Weightlifting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1673677086.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Weightlifting_final = Weightlifting_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Water Polo

In [120]:
Polo_df = data_df[data_df['Sport'] == "Water Polo"]
Polo_df = Polo_df.dropna()

In [121]:
Polo_dummies = pd.get_dummies(Polo_df)
Polo_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,NOC_CHN,NOC_CUB,NOC_CZE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
58,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
121,True,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
189,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [122]:
x_train = Polo_dummies.drop(columns=['Medal'])
y_train = Polo_dummies['Medal']

x_test = Polo_dummies.drop(columns=['Medal'])
y_test = Polo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

C:\Users\14043\Downloads\Newfolder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8627450980392157

In [123]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Polo_df)

array([0.17200241, 0.18422608, 0.67229382, ..., 0.86621261, 0.00837879,
       0.30003566])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
58,NLD,Water Polo,False,0,0,0,3,8,0
121,ESP,Water Polo,True,0,0,0,2,8,0
189,ITA,Water Polo,False,0,1,0,2,8,0
251,EGY,Water Polo,False,0,0,0,5,7,0
267,EGY,Water Polo,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
203275,RUS,Water Polo,False,0,0,0,2,6,0
203613,ESP,Water Polo,False,0,0,0,3,7,0
203630,RUS,Water Polo,False,0,0,0,2,6,0
203631,KAZ,Water Polo,False,0,0,0,4,6,0


In [124]:
Polo_df['Predictions'] = y_pred.tolist()
Polo_final = Polo_df.drop_duplicates(subset=['NOC'], keep='last')
Polo_final = Polo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1686184237.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Polo_final = Polo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [125]:
I = pd.merge(Weightlifting_final, Polo_final, how='outer', on=['NOC', 'NOC'])
I.rename(columns = {'Predictions_x':'Weightlifting', 'Predictions_y':'Water Polo'}, inplace = True)
I = I.drop(["Sport_x","Sport_y"], 1)
I

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1958365582.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  I = I.drop(["Sport_x","Sport_y"], 1)


,NOC,Weightlifting,Water Polo
0,SAU,0.199698,NaN
1,GHA,0.055781,NaN
2,ZWE,0.018348,NaN
3,SLE,0.060120,NaN
4,LBN,0.199698,NaN
...,...,...,...
121,CZE,0.144099,0.016672
122,ESP,0.338532,0.479836
123,POL,0.656120,NaN
124,SRB,NaN,0.988492


Archery

In [126]:
Archery_df = data_df[data_df['Sport'] == "Archery"]
Archery_df = Archery_df.dropna()

In [127]:
Archery_dummies = pd.get_dummies(Archery_df)
Archery_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGD,NOC_BGR,NOC_BLR,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
206,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
521,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
764,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [128]:
x_train = Archery_dummies.drop(columns=['Medal'])
y_train = Archery_dummies['Medal']

x_test = Archery_dummies.drop(columns=['Medal'])
y_test = Archery_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9129082426127527

In [129]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Archery_df)

array([0.01310072, 0.05049789, 0.51941128, ..., 0.82805867, 0.82805867,
       0.21284847])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
206,MAR,Archery,False,0,0,0,4,7,0
521,EGY,Archery,False,0,0,0,5,7,0
764,RUS,Archery,False,0,0,0,2,7,0
765,RUS,Archery,False,0,0,0,2,7,0
845,IRN,Archery,False,0,0,0,2,7,0
...,...,...,...,...,...,...,...,...,...
202043,CHN,Archery,False,0,0,8,5,7,1
202044,CHN,Archery,True,0,0,8,5,7,1
202045,CHN,Archery,True,1,0,7,4,7,1
202046,CHN,Archery,True,1,0,7,4,7,1


In [130]:
Archery_df['Predictions'] = y_pred.tolist()
Archery_final = Archery_df.drop_duplicates(subset=['NOC'], keep='last')
Archery_final = Archery_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/4291309299.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Archery_final = Archery_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Diving

In [131]:
Diving_df = data_df[data_df['Sport'] == "Diving"]
Diving_df = Diving_df.dropna()

In [132]:
Diving_dummies = pd.get_dummies(Diving_df)
Diving_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
108,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
235,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
538,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [133]:
x_train = Diving_dummies.drop(columns=['Medal'])
y_train = Diving_dummies['Medal']

x_test = Diving_dummies.drop(columns=['Medal'])
y_test = Diving_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8816234498308907

In [134]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Diving_df)

array([0.03381947, 0.04389726, 0.51633471, ..., 0.25326653, 0.25326653,
       0.25326653])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
108,CUB,Diving,False,0,0,0,2,8,0
235,EGY,Diving,False,0,0,0,5,7,0
538,CAN,Diving,False,0,2,0,1,8,0
539,CAN,Diving,False,0,2,0,1,8,0
540,CAN,Diving,True,0,2,0,1,8,0
...,...,...,...,...,...,...,...,...,...
202802,UKR,Diving,False,0,0,0,3,7,0
202803,UKR,Diving,False,0,0,0,3,7,0
202804,UKR,Diving,False,0,0,0,3,7,0
202805,UKR,Diving,True,0,0,0,3,7,0


In [135]:
Diving_df['Predictions'] = y_pred.tolist()
Diving_final = Diving_df.drop_duplicates(subset=['NOC'], keep='last')
Diving_final = Diving_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1671132766.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Diving_final = Diving_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [136]:
J = pd.merge(Archery_final, Diving_final, how='outer', on=['NOC', 'NOC'])
J.rename(columns = {'Predictions_x':'Archery', 'Predictions_y':'Diving'}, inplace = True)
J = J.drop(["Sport_x","Sport_y"], 1)
J

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2986169119.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  J = J.drop(["Sport_x","Sport_y"], 1)


,NOC,Archery,Diving
0,MAR,0.013101,NaN
1,IRQ,0.050498,NaN
2,JOR,0.050498,NaN
3,ROU,0.013101,0.043897
4,DOM,0.031375,0.037727
...,...,...,...
93,BHS,NaN,0.043897
94,PER,NaN,0.043897
95,SYR,NaN,0.043897
96,ECU,NaN,0.037727


Tennis

In [137]:
Tennis_df = data_df[data_df['Sport'] == "Tennis"]
Tennis_df = Tennis_df.dropna()

In [138]:
Tennis_dummies = pd.get_dummies(Tennis_df)
Tennis_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BEN,NOC_BGR,NOC_BHS,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
123,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
446,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1008,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [139]:
x_train = Tennis_dummies.drop(columns=['Medal'])
y_train = Tennis_dummies['Medal']

x_test = Tennis_dummies.drop(columns=['Medal'])
y_test = Tennis_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9209989806320081

In [140]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Tennis_df)

array([0.06810516, 0.03028908, 0.02199167, ..., 0.67154737, 0.67154737,
       0.67154737])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
123,MEX,Tennis,False,0,0,0,2,7,0
446,NGA,Tennis,False,0,0,0,6,4,0
1008,BRA,Tennis,False,0,0,1,2,6,0
1178,ZAF,Tennis,False,0,0,0,4,5,0
1607,USA,Tennis,True,1,1,1,2,8,5
...,...,...,...,...,...,...,...,...,...
203920,BLR,Tennis,False,0,0,0,2,7,0
203921,BLR,Tennis,False,0,0,0,2,7,0
203926,RUS,Tennis,True,0,0,0,2,7,0
203927,RUS,Tennis,False,0,0,0,2,7,0


In [141]:
Tennis_df['Predictions'] = y_pred.tolist()
Tennis_final = Tennis_df.drop_duplicates(subset=['NOC'], keep='last')
Tennis_final = Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Tennis_final

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3496524252.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Tennis_final = Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
3362,MDA,Tennis,0.168556
7152,SLV,Tennis,0.042128
15122,LTU,Tennis,0.042128
24797,TUR,Tennis,0.117699
25595,BOL,Tennis,0.030831
...,...,...,...
203567,CZE,Tennis,0.575483
203569,SVK,Tennis,0.036352
203757,COL,Tennis,0.117699
203921,BLR,Tennis,0.538296


Table Tennis

In [142]:
Table_Tennis_df = data_df[data_df['Sport'] == "Table Tennis"]
Table_Tennis_df = Table_Tennis_df.dropna()

In [143]:
Table_Tennis_dummies = pd.get_dummies(Table_Tennis_df)
Table_Tennis_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BIH,NOC_BLR,NOC_BRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
369,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
370,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
371,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [144]:
x_train = Table_Tennis_dummies.drop(columns=['Medal'])
y_train = Table_Tennis_dummies['Medal']

x_test = Table_Tennis_dummies.drop(columns=['Medal'])
y_test = Table_Tennis_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9215572341661824

In [145]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Table_Tennis_df)

array([0.00587529, 0.00587529, 0.00587529, ..., 0.00535769, 0.00567739,
       0.00663938])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
369,EGY,Table Tennis,False,0,0,0,5,7,0
370,EGY,Table Tennis,False,0,0,0,5,7,0
371,EGY,Table Tennis,False,0,0,0,5,7,0
611,NGA,Table Tennis,False,0,0,1,4,5,0
962,IND,Table Tennis,False,0,0,7,6,6,0
...,...,...,...,...,...,...,...,...,...
202757,USA,Table Tennis,False,1,1,1,2,8,5
202758,USA,Table Tennis,False,0,1,1,1,8,6
202759,USA,Table Tennis,False,0,1,1,1,8,6
202760,USA,Table Tennis,False,0,2,1,1,8,5


In [146]:
Table_Tennis_df['Predictions'] = y_pred.tolist()
Table_Tennis_final = Table_Tennis_df.drop_duplicates(subset=['NOC'], keep='last')
Table_Tennis_final = Table_Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/959563229.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Table_Tennis_final = Table_Tennis_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [147]:
K = pd.merge(Tennis_final, Table_Tennis_final, how='outer', on=['NOC', 'NOC'])
K.rename(columns = {'Predictions_x':'Tennis', 'Predictions_y':'Table Tennis'}, inplace = True)
K = K.drop(["Sport_x","Sport_y"], 1)
K

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2974582299.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  K = K.drop(["Sport_x","Sport_y"], 1)


,NOC,Tennis,Table Tennis
0,MDA,0.168556,NaN
1,SLV,0.042128,NaN
2,LTU,0.042128,0.006639
3,TUR,0.117699,0.006639
4,BOL,0.030831,NaN
...,...,...,...
106,VUT,NaN,0.003067
107,HKG,NaN,0.107393
108,COG,NaN,0.007957
109,FJI,NaN,0.006639


Badminton

In [148]:
Badminton_df = data_df[data_df['Sport'] == "Badminton"]
Badminton_df = Badminton_df.dropna()

In [149]:
Badminton_dummies = pd.get_dummies(Badminton_df)
Badminton_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_BRN,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
26,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
593,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
594,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [150]:
x_train = Badminton_dummies.drop(columns=['Medal'])
y_train = Badminton_dummies['Medal']

x_test = Badminton_dummies.drop(columns=['Medal'])
y_test = Badminton_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8830242510699001

In [151]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Badminton_df)

array([0.03054617, 0.22672519, 0.3621166 , ..., 0.02280393, 0.02280393,
       0.01243125])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
26,FIN,Badminton,False,0,1,0,1,8,0
593,ESP,Badminton,False,0,1,0,2,8,0
594,ESP,Badminton,False,0,1,0,1,8,0
595,ESP,Badminton,False,0,1,0,1,8,0
1239,GBR,Badminton,False,1,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
203867,RUS,Badminton,False,0,0,1,2,6,0
203868,RUS,Badminton,False,0,0,0,2,6,0
203952,DEU,Badminton,False,0,2,0,2,8,1
203953,DEU,Badminton,False,0,2,0,2,8,1


In [152]:
Badminton_df['Predictions'] = y_pred.tolist()
Badminton_final = Badminton_df.drop_duplicates(subset=['NOC'], keep='last')
Badminton_final = Badminton_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1196665366.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Badminton_final = Badminton_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Modern Pentathlon

In [153]:
Pentathlon_df = data_df[data_df['Sport'] == "Modern Pentathlon"]
Pentathlon_df = Pentathlon_df.dropna()

In [154]:
Pentathlon_dummies = pd.get_dummies(Pentathlon_df)
Pentathlon_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BHR,NOC_BLR,NOC_BRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
355,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
356,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
357,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [155]:
x_train = Pentathlon_dummies.drop(columns=['Medal'])
y_train = Pentathlon_dummies['Medal']

x_test = Pentathlon_dummies.drop(columns=['Medal'])
y_test = Pentathlon_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9207135777998018

In [156]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Pentathlon_df)

array([0.03069475, 0.03808506, 0.03808506, ..., 0.16514419, 0.05436268,
       0.05436268])

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
355,EGY,Modern Pentathlon,False,0,0,0,5,7,0
356,EGY,Modern Pentathlon,False,0,0,0,5,6,0
357,EGY,Modern Pentathlon,False,0,0,0,5,6,0
358,EGY,Modern Pentathlon,False,0,0,0,5,6,0
359,EGY,Modern Pentathlon,False,0,0,0,5,6,0
...,...,...,...,...,...,...,...,...,...
202201,CHN,Modern Pentathlon,False,0,0,7,4,8,3
202814,BLR,Modern Pentathlon,False,0,0,0,2,7,0
202989,DEU,Modern Pentathlon,False,0,1,0,2,8,1
203310,AUT,Modern Pentathlon,False,0,0,0,3,7,0


In [157]:
Pentathlon_df['Predictions'] = y_pred.tolist()
Pentathlon_final = Pentathlon_df.drop_duplicates(subset=['NOC'], keep='last')
Pentathlon_final = Pentathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1043166688.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Pentathlon_final = Pentathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [158]:
L = pd.merge(Badminton_final, Pentathlon_final, how='outer', on=['NOC', 'NOC'])
L.rename(columns = {'Predictions_x':'Badminton', 'Predictions_y':'Modern Pentathlon'}, inplace = True)
L = L.drop(["Sport_x","Sport_y"], 1)
L

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1863086147.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  L = L.drop(["Sport_x","Sport_y"], 1)


,NOC,Badminton,Modern Pentathlon
0,ROU,0.042543,0.070224
1,ITA,0.016013,0.298572
2,SYC,0.042543,NaN
3,UGA,0.049880,NaN
4,EGY,0.063613,0.038085
...,...,...,...
69,LVA,NaN,0.561855
70,KAZ,NaN,0.490180
71,KGZ,NaN,0.166424
72,CUB,NaN,0.097990


Synchronized Swimming

In [159]:
S_Swimming_df = data_df[data_df['Sport'] == "Synchronized Swimming"]
S_Swimming_df = S_Swimming_df.dropna()

In [160]:
S_Swimming_dummies = pd.get_dummies(S_Swimming_df)
S_Swimming_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
214,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
215,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
216,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [161]:
x_train = S_Swimming_dummies.drop(columns=['Medal'])
y_train = S_Swimming_dummies['Medal']

x_test = S_Swimming_dummies.drop(columns=['Medal'])
y_test = S_Swimming_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.905638665132336

In [162]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,S_Swimming_df)

array([2.00261443e-02, 2.00261443e-02, 2.00261443e-02, 2.00261443e-02,
       2.00261443e-02, 2.00261443e-02, 2.00261443e-02, 2.00261443e-02,
       2.00261443e-02, 2.00261443e-02, 2.00261443e-02, 3.24011263e-02,
       1.27676669e-02, 3.99107352e-02, 3.43912691e-02, 1.57689525e-01,
       1.57689525e-01, 1.57689525e-01, 2.00261443e-02, 7.35597703e-02,
       2.00261443e-02, 2.00261443e-02, 7.47813504e-01, 2.67601572e-04,
       2.67601572e-04, 2.00261443e-02, 2.00261443e-02, 2.00261443e-02,
       3.70470089e-04, 3.56934447e-02, 3.56934447e-02, 3.11355527e-02,
       3.24011263e-02, 2.00261443e-02, 2.00261443e-02, 1.96630907e-04,
       1.27676669e-02, 1.27676669e-02, 1.27676669e-02, 5.25443127e-01,
       5.60500281e-01, 5.60500281e-01, 2.00261443e-02, 2.00261443e-02,
       4.83251100e-02, 2.00261443e-02, 1.27676669e-02, 5.25443127e-01,
       5.25443127e-01, 4.56980234e-01, 5.25251500e-01, 2.00261443e-02,
       5.91536799e-02, 5.25443127e-01, 5.25443127e-01, 5.60500281e-01,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
214,EGY,Synchronized Swimming,False,0,0,0,5,7,0
215,EGY,Synchronized Swimming,False,0,0,0,5,7,0
216,EGY,Synchronized Swimming,False,0,0,0,5,7,0
246,EGY,Synchronized Swimming,False,0,0,0,5,7,0
247,EGY,Synchronized Swimming,False,0,0,0,5,7,0
...,...,...,...,...,...,...,...,...,...
202196,CHN,Synchronized Swimming,True,1,0,7,4,7,1
202249,CHN,Synchronized Swimming,False,0,0,8,7,7,0
202444,KAZ,Synchronized Swimming,False,0,0,0,4,7,0
203935,NLD,Synchronized Swimming,False,0,1,0,2,8,0


In [163]:
S_Swimming_df['Predictions'] = y_pred.tolist()
S_Swimming_final = S_Swimming_df.drop_duplicates(subset=['NOC'], keep='last')
S_Swimming_final = S_Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
S_Swimming_final

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1902890597.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  S_Swimming_final = S_Swimming_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
6909,COL,Synchronized Swimming,0.020026
78326,SWE,Synchronized Swimming,0.031136
97996,SVK,Synchronized Swimming,0.020026
99981,FIN,Synchronized Swimming,0.031136
112650,HUN,Synchronized Swimming,0.017297
153259,CUB,Synchronized Swimming,0.020026
154914,VEN,Synchronized Swimming,0.012768
156097,NZL,Synchronized Swimming,0.012768
157764,ARG,Synchronized Swimming,0.073560
161006,DEU,Synchronized Swimming,0.087332


Baseball

In [164]:
Baseball_df = data_df[data_df['Sport'] == "Baseball"]
Baseball_df = Baseball_df.dropna()

In [165]:
Baseball_dummies = pd.get_dummies(Baseball_df)
Baseball_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_CAN,NOC_CHN,NOC_CUB,NOC_DOM,NOC_ESP,NOC_GRC,NOC_ITA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
505,False,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
506,False,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
578,True,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [166]:
x_train = Baseball_dummies.drop(columns=['Medal'])
y_train = Baseball_dummies['Medal']

x_test = Baseball_dummies.drop(columns=['Medal'])
y_test = Baseball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9463087248322147

In [167]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Baseball_df)

array([2.06342082e-01, 3.40526006e-02, 9.68661504e-01, 2.30153528e-01,
       5.67325154e-03, 2.54242927e-02, 6.47446390e-04, 4.68455100e-03,
       9.68609559e-01, 9.68661504e-01, 9.81259382e-01, 9.81259382e-01,
       9.99967152e-01, 2.06342082e-01, 3.07649865e-04, 5.67325154e-03,
       5.67325154e-03, 2.30153528e-01, 9.27068493e-01, 6.45213385e-01,
       9.69097049e-01, 1.00267229e-01, 6.03779945e-01, 9.68609559e-01,
       3.40526006e-02, 3.07649865e-04, 9.99967152e-01, 3.40526006e-02,
       3.40526006e-02, 3.07649865e-04, 6.22037732e-02, 9.69097049e-01,
       9.81259382e-01, 6.22037732e-02, 3.07649865e-04, 4.54137183e-02,
       3.07649865e-04, 6.45213385e-01, 3.79151612e-03, 5.51905470e-03,
       3.87547190e-02, 6.47446390e-04, 2.54242927e-02, 2.54242927e-02,
       6.47446390e-04, 1.00267229e-01, 5.51905470e-03, 9.69097049e-01,
       6.22037732e-02, 1.00267229e-01, 6.50601374e-03, 4.54137183e-02,
       2.54242927e-02, 6.47446390e-04, 9.99967152e-01, 5.67325154e-03,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
505,JPN,Baseball,False,0,1,0,1,8,3
506,JPN,Baseball,False,0,1,0,1,8,1
578,USA,Baseball,True,0,1,1,1,8,6
1216,USA,Baseball,False,0,1,1,2,7,5
1439,ZAF,Baseball,False,0,0,0,4,5,0
...,...,...,...,...,...,...,...,...,...
201091,NIC,Baseball,False,0,0,0,4,7,0
201419,GRC,Baseball,False,0,1,0,2,8,0
202009,CHN,Baseball,False,1,0,7,4,7,1
202064,CHN,Baseball,False,1,0,7,4,7,1


In [168]:
Baseball_df['Predictions'] = y_pred.tolist()
Baseball_final = Baseball_df.drop_duplicates(subset=['NOC'], keep='last')
Baseball_final = Baseball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
Baseball_final

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1263594060.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Baseball_final = Baseball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
186125,ESP,Baseball,0.062204
187383,ZAF,Baseball,0.005673
188471,ITA,Baseball,0.005519
188746,PRI,Baseball,0.000308
188969,CUB,Baseball,0.999967
190122,DOM,Baseball,0.000308
193818,CAN,Baseball,0.045414
195652,AUS,Baseball,0.603780
197203,NLD,Baseball,0.025424
199761,KOR,Baseball,0.999055


In [169]:
M = pd.merge(S_Swimming_final, Baseball_final, how='outer', on=['NOC', 'NOC'])
M.rename(columns = {'Predictions_x':'Sychronized Swimming', 'Predictions_y':'Baseball'}, inplace = True)
M = M.drop(["Sport_x","Sport_y"], 1)
M

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/184479326.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  M = M.drop(["Sport_x","Sport_y"], 1)


,NOC,Sychronized Swimming,Baseball
0,COL,0.020026,NaN
1,SWE,0.031136,NaN
2,SVK,0.020026,NaN
3,FIN,0.031136,NaN
4,HUN,0.017297,NaN
5,CUB,0.020026,0.999967
6,VEN,0.012768,NaN
7,NZL,0.012768,NaN
8,ARG,0.073560,NaN
9,DEU,0.087332,NaN


Rhythmic Gymnastics

In [170]:
R_Gymnastics_df = data_df[data_df['Sport'] == "Rhythmic Gymnastics"]
R_Gymnastics_df = R_Gymnastics_df.dropna()

In [171]:
R_Gymnastics_dummies = pd.get_dummies(R_Gymnastics_df)
R_Gymnastics_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BGR,NOC_BLR,NOC_BRA,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
527,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
759,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
935,False,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [172]:
x_train = R_Gymnastics_dummies.drop(columns=['Medal'])
y_train = R_Gymnastics_dummies['Medal']

x_test = R_Gymnastics_dummies.drop(columns=['Medal'])
y_test = R_Gymnastics_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8769716088328076

In [173]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,R_Gymnastics_df)

array([0.43133903, 0.07217754, 0.78652983, 0.47820774, 0.47820774,
       0.47820774, 0.45208453, 0.02389838, 0.00405976, 0.75058436,
       0.34400949, 0.98029102, 0.07217754, 0.70935478, 0.07217754,
       0.45208453, 0.75058436, 0.28530136, 0.28530136, 0.03246017,
       0.6105615 , 0.07217754, 0.03246017, 0.69284714, 0.04221269,
       0.75058436, 0.07217754, 0.6105615 , 0.07217754, 0.25723802,
       0.034169  , 0.98738284, 0.96507234, 0.75058436, 0.98738284,
       0.98738284, 0.13618215, 0.03246017, 0.03535943, 0.03246017,
       0.07217754, 0.5572827 , 0.03528003, 0.00747236, 0.36891258,
       0.38392233, 0.07217754, 0.36891258, 0.03535943, 0.98029102,
       0.07217754, 0.03246017, 0.04221269, 0.070868  , 0.54145198,
       0.5572827 , 0.5572827 , 0.98029102, 0.98029102, 0.07217754,
       0.6105615 , 0.54145198, 0.96507234, 0.07217754, 0.07217754,
       0.0523099 , 0.03246017, 0.75058436, 0.75058436, 0.04032281,
       0.6105615 , 0.80979547, 0.07217754, 0.6105615 , 0.03246

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
527,ESP,Rhythmic Gymnastics,False,0,1,0,2,8,0
759,GEO,Rhythmic Gymnastics,False,0,0,0,4,7,0
935,ESP,Rhythmic Gymnastics,False,1,0,0,2,8,0
961,ESP,Rhythmic Gymnastics,False,0,1,0,1,8,0
1783,ESP,Rhythmic Gymnastics,False,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
202970,RUS,Rhythmic Gymnastics,True,0,0,0,2,6,0
203071,USA,Rhythmic Gymnastics,False,1,0,1,2,7,7
203465,NOR,Rhythmic Gymnastics,False,0,0,0,2,8,0
203608,UKR,Rhythmic Gymnastics,False,0,0,0,2,7,0


In [174]:
R_Gymnastics_df['Predictions'] = y_pred.tolist()
R_Gymnastics_final = R_Gymnastics_df.drop_duplicates(subset=['NOC'], keep='last')
R_Gymnastics_final = R_Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)
R_Gymnastics_final

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2322458286.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  R_Gymnastics_final = R_Gymnastics_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


,NOC,Sport,Predictions
14839,SWE,Rhythmic Gymnastics,0.136182
53785,DNK,Rhythmic Gymnastics,0.046236
82268,PRT,Rhythmic Gymnastics,0.072178
88051,EST,Rhythmic Gymnastics,0.076579
90843,LTU,Rhythmic Gymnastics,0.072178
121737,CPV,Rhythmic Gymnastics,0.072178
145369,AUS,Rhythmic Gymnastics,0.043892
157925,ZAF,Rhythmic Gymnastics,0.109050
160902,BRA,Rhythmic Gymnastics,0.044040
162979,FRA,Rhythmic Gymnastics,0.059379


Taekwondo

In [175]:
Taekwondo_df = data_df[data_df['Sport'] == "Taekwondo"]
Taekwondo_df = Taekwondo_df.dropna()

In [176]:
Taekwondo_dummies = pd.get_dummies(Taekwondo_df)
Taekwondo_dummies.head(3)

,Medal,Host,NOC_AFG,NOC_ARE,NOC_ARG,NOC_ARM,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
92,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
101,True,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
213,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [177]:
x_train = Taekwondo_dummies.drop(columns=['Medal'])
y_train = Taekwondo_dummies['Medal']

x_test = Taekwondo_dummies.drop(columns=['Medal'])
y_test = Taekwondo_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.7979966611018364

In [178]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Taekwondo_df)

array([0.2484668 , 0.5142375 , 0.2484668 , 0.76553013, 0.15771783,
       0.15771783, 0.406888  , 0.406888  , 0.16483402, 0.41253042,
       0.2484668 , 0.57545659, 0.2484668 , 0.23305483, 0.406888  ,
       0.406888  , 0.15771783, 0.25546949, 0.15771783, 0.55102483,
       0.51912629, 0.60976064, 0.10356875, 0.23305483, 0.22853876,
       0.2484668 , 0.13085924, 0.16483402, 0.60976064, 0.25546949,
       0.08476774, 0.51684818, 0.58373127, 0.16483402, 0.29321219,
       0.68886933, 0.74149326, 0.406888  , 0.27879378, 0.41113236,
       0.63576943, 0.51821268, 0.23305483, 0.6247539 , 0.08476774,
       0.15771783, 0.28111006, 0.5662077 , 0.5662077 , 0.52796674,
       0.67483535, 0.76040771, 0.18990202, 0.5142375 , 0.20003404,
       0.11225064, 0.11225064, 0.23305483, 0.23305483, 0.23305483,
       0.23305483, 0.52031865, 0.25942007, 0.11225064, 0.65270235,
       0.23305483, 0.51507853, 0.52600345, 0.68972996, 0.38065049,
       0.38065049, 0.1867906 , 0.1867906 , 0.11225064, 0.11225

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
92,EGY,Taekwondo,False,0,0,0,5,7,0
101,AZE,Taekwondo,True,0,0,0,4,7,0
213,EGY,Taekwondo,False,0,0,0,5,7,0
230,USA,Taekwondo,True,0,1,1,1,8,6
475,KGZ,Taekwondo,False,0,0,0,6,7,0
...,...,...,...,...,...,...,...,...,...
203239,AZE,Taekwondo,False,0,0,0,4,7,0
203240,AZE,Taekwondo,False,0,0,0,4,7,0
203565,MAR,Taekwondo,False,0,0,0,4,7,0
203566,MAR,Taekwondo,False,0,0,0,4,7,0


In [179]:
Taekwondo_df['Predictions'] = y_pred.tolist()
Taekwondo_final = Taekwondo_df.drop_duplicates(subset=['NOC'], keep='last')
Taekwondo_final = Taekwondo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/4211559542.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Taekwondo_final = Taekwondo_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [180]:
N = pd.merge(R_Gymnastics_final, Taekwondo_final, how='outer', on=['NOC', 'NOC'])
N.rename(columns = {'Predictions_x':'Rhythmic Gymnastics', 'Predictions_y':'Taekwondo'}, inplace = True)
N = N.drop(["Sport_x","Sport_y"], 1)
N

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/1912829512.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  N = N.drop(["Sport_x","Sport_y"], 1)


,NOC,Rhythmic Gymnastics,Taekwondo
0,SWE,0.136182,0.175473
1,DNK,0.046236,0.255469
2,PRT,0.072178,0.178303
3,EST,0.076579,NaN
4,LTU,0.072178,NaN
...,...,...,...
104,THA,NaN,0.667139
105,ARM,NaN,0.233055
106,GTM,NaN,0.293212
107,MAR,NaN,0.112251


Beach Volleyball

In [181]:
B_Volleyball_df = data_df[data_df['Sport'] == "Beach Volleyball"]
B_Volleyball_df = B_Volleyball_df.dropna()

In [182]:
B_Volleyball_dummies = pd.get_dummies(B_Volleyball_df)
B_Volleyball_dummies.head(3)

,Medal,Host,NOC_AGO,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGR,NOC_BRA,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
816,False,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1912,False,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1913,True,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [183]:
x_train = B_Volleyball_dummies.drop(columns=['Medal'])
y_train = B_Volleyball_dummies['Medal']

x_test = B_Volleyball_dummies.drop(columns=['Medal'])
y_test = B_Volleyball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8714285714285714

In [184]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,B_Volleyball_df)

array([0.00297108, 0.48509129, 0.48509129, 0.07255478, 0.86538825,
       0.90225253, 0.07255478, 0.07255478, 0.40520827, 0.86538825,
       0.86538825, 0.02740387, 0.64710808, 0.07994222, 0.07994222,
       0.07994222, 0.12124829, 0.27685897, 0.02740387, 0.25758572,
       0.29999674, 0.29999674, 0.04032418, 0.08307479, 0.08307479,
       0.90225253, 0.45958632, 0.0692421 , 0.07255478, 0.07255478,
       0.0692421 , 0.64710808, 0.90225253, 0.86538825, 0.86538825,
       0.02740387, 0.07994222, 0.09161805, 0.07255478, 0.86538825,
       0.04458771, 0.04458771, 0.03034451, 0.03034451, 0.07103435,
       0.30866913, 0.25758572, 0.78327729, 0.78327729, 0.48509129,
       0.21329629, 0.48509129, 0.41806533, 0.41806533, 0.44379826,
       0.7673458 , 0.04032418, 0.04032418, 0.04032418, 0.54799025,
       0.54799025, 0.25758572, 0.3677313 , 0.36893001, 0.18094434,
       0.02740387, 0.48509129, 0.48509129, 0.07255478, 0.20912678,
       0.40520827, 0.8726343 , 0.07255478, 0.07255478, 0.07255

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
816,AGO,Beach Volleyball,False,0,0,0,5,5,0
1912,DEU,Beach Volleyball,False,0,1,0,2,8,1
1913,DEU,Beach Volleyball,True,0,1,0,2,8,1
3764,CRI,Beach Volleyball,False,0,0,0,2,8,0
3948,BRA,Beach Volleyball,True,0,0,1,1,7,0
...,...,...,...,...,...,...,...,...,...
202179,CHN,Beach Volleyball,True,1,0,7,4,7,1
202180,CHN,Beach Volleyball,False,0,0,7,4,7,2
203411,ARG,Beach Volleyball,False,0,0,0,0,7,0
203768,CHE,Beach Volleyball,False,0,3,0,2,8,0


In [185]:
B_Volleyball_df['Predictions'] = y_pred.tolist()
B_Volleyball_final = B_Volleyball_df.drop_duplicates(subset=['NOC'], keep='last')
B_Volleyball_final = B_Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3648103712.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  B_Volleyball_final = B_Volleyball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Triathlon

In [186]:
Triathlon_df = data_df[data_df['Sport'] == "Triathlon"]
Triathlon_df = Triathlon_df.dropna()

In [187]:
Triathlon_dummies = pd.get_dummies(Triathlon_df)
Triathlon_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_AZE,NOC_BEL,NOC_BRA,NOC_BRB,NOC_CAN,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
831,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
901,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
902,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [188]:
x_train = Triathlon_dummies.drop(columns=['Medal'])
y_train = Triathlon_dummies['Medal']

x_test = Triathlon_dummies.drop(columns=['Medal'])
y_test = Triathlon_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.941972920696325

In [189]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Triathlon_df)

array([3.76758911e-02, 3.76758911e-02, 3.76758911e-02, 4.68789698e-04,
       4.66132616e-01, 5.30353389e-01, 3.76758911e-02, 3.76758911e-02,
       5.27946145e-01, 4.66132616e-01, 8.16776632e-03, 1.40697516e-01,
       1.40697516e-01, 7.57410397e-01, 4.68537761e-01, 8.85132655e-02,
       7.76371798e-01, 7.54878264e-01, 1.88100515e-01, 3.76758911e-02,
       9.90981886e-03, 1.74753672e-01, 9.41958593e-02, 8.88522715e-02,
       5.42289204e-01, 1.44155071e-01, 2.40416827e-03, 1.87789773e-03,
       8.16412021e-01, 5.26935783e-01, 3.76825551e-01, 8.57441367e-01,
       2.07940454e-01, 4.48659183e-01, 4.48659183e-01, 4.65693895e-02,
       5.26935783e-01, 9.90981886e-03, 4.65693895e-02, 2.40416827e-03,
       4.38842610e-01, 4.68537761e-01, 5.30353389e-01, 1.07298109e-01,
       1.07298109e-01, 8.83143796e-03, 3.33639409e-02, 5.27946145e-01,
       4.66132616e-01, 5.80317174e-01, 8.67209010e-02, 2.37002132e-02,
       2.28849040e-01, 6.91183673e-02, 1.88100515e-01, 8.57441367e-01,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
831,RUS,Triathlon,False,0,0,0,2,7,0
901,RUS,Triathlon,False,0,0,0,2,7,0
902,RUS,Triathlon,False,0,0,0,2,7,0
1078,JPN,Triathlon,False,0,2,0,0,8,1
1685,AUT,Triathlon,False,0,2,0,3,8,0
...,...,...,...,...,...,...,...,...,...
201588,BEL,Triathlon,False,0,2,0,0,8,0
201620,USA,Triathlon,False,0,1,1,1,8,6
201646,CZE,Triathlon,False,0,0,0,2,7,0
202244,CHN,Triathlon,False,1,0,7,4,7,1


In [190]:
Triathlon_df['Predictions'] = y_pred.tolist()
Triathlon_final = Triathlon_df.drop_duplicates(subset=['NOC'], keep='last')
Triathlon_final = Triathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/2330985694.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Triathlon_final = Triathlon_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [191]:
O = pd.merge(B_Volleyball_final, Triathlon_final, how='outer', on=['NOC', 'NOC'])
O.rename(columns = {'Predictions_x':'Beach Volleyball', 'Predictions_y':'Triathlon'}, inplace = True)
O = O.drop(["Sport_x","Sport_y"], 1)
O

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3167638783.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  O = O.drop(["Sport_x","Sport_y"], 1)


,NOC,Beach Volleyball,Triathlon
0,TUN,0.027404,NaN
1,CRI,0.072555,0.023700
2,AGO,0.002971,NaN
3,CHL,0.083075,0.033364
4,NZL,0.040324,0.069118
5,EGY,0.002971,NaN
6,GBR,0.062799,0.857441
7,PRI,0.083075,NaN
8,SWE,0.044588,0.788051
9,POL,0.027404,0.023700


Softball

In [192]:
Softball_df = data_df[data_df['Sport'] == "Softball"]
Softball_df = Softball_df.dropna()

In [193]:
Softball_dummies = pd.get_dummies(Softball_df)
Softball_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_CAN,NOC_CHN,NOC_CUB,NOC_GRC,NOC_ITA,NOC_JPN,NOC_NLD,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
205,True,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2088,False,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
3904,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [194]:
x_train = Softball_dummies.drop(columns=['Medal'])
y_train = Softball_dummies['Medal']

x_test = Softball_dummies.drop(columns=['Medal'])
y_test = Softball_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9686192468619247

In [195]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Softball_df)

array([9.98699835e-01, 1.47821717e-04, 2.26199276e-02, 9.92782596e-01,
       9.92782596e-01, 9.71447679e-01, 1.47821717e-04, 9.99678094e-01,
       5.92688266e-01, 5.23702608e-03, 1.25507123e-01, 9.57386245e-01,
       1.94361833e-02, 2.26199276e-02, 1.47821717e-04, 3.70126609e-02,
       1.47821717e-04, 1.47821717e-04, 2.26199276e-02, 1.94361833e-02,
       1.94361833e-02, 1.94361833e-02, 3.70126609e-02, 3.70126609e-02,
       9.41727708e-03, 1.83134840e-06, 2.26362374e-03, 2.26362374e-03,
       9.52011630e-01, 9.99678094e-01, 9.99678094e-01, 9.98699835e-01,
       8.07516572e-02, 2.26199276e-02, 1.94361833e-02, 9.71447679e-01,
       9.52011630e-01, 3.70126609e-02, 9.41727708e-03, 9.92782596e-01,
       9.92782596e-01, 9.99678094e-01, 1.47821717e-04, 1.94361833e-02,
       9.99678094e-01, 9.99678094e-01, 9.98699835e-01, 9.92782596e-01,
       3.70126609e-02, 1.47821717e-04, 1.94361833e-02, 2.26199276e-02,
       1.47821717e-04, 1.47821717e-04, 5.92688266e-01, 2.92724982e-02,
      

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
205,USA,Softball,True,0,2,1,1,8,5
2088,ITA,Softball,False,0,0,0,3,8,0
3904,VEN,Softball,False,0,0,0,1,7,0
4121,AUS,Softball,True,1,1,0,1,8,0
4122,AUS,Softball,True,0,1,0,1,8,0
...,...,...,...,...,...,...,...,...,...
202090,CHN,Softball,False,0,0,8,5,7,1
202240,CHN,Softball,False,0,0,8,5,7,0
202668,CHN,Softball,False,0,0,8,5,7,0
202676,CHN,Softball,False,0,0,8,5,7,1


In [196]:
Softball_df['Predictions'] = y_pred.tolist()
Softball_final = Softball_df.drop_duplicates(subset=['NOC'], keep='last')
Softball_final = Softball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3040282946.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Softball_final = Softball_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Rugby Sevens

In [197]:
Rugby_df = data_df[data_df['Sport'] == "Rugby Sevens"]
Rugby_df = Rugby_df.dropna()

In [198]:
Rugby_dummies = pd.get_dummies(Rugby_df)
Rugby_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_BRA,NOC_CAN,NOC_COL,NOC_ESP,NOC_FJI,NOC_FRA,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
605,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
950,False,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
951,False,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [199]:
x_train = Rugby_dummies.drop(columns=['Medal'])
y_train = Rugby_dummies['Medal']

x_test = Rugby_dummies.drop(columns=['Medal'])
y_test = Rugby_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8361204013377926

In [200]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Rugby_df)

array([0.06256206, 0.11314278, 0.11314278, 0.06256206, 0.96192433,
       0.96192433, 0.0601974 , 0.00710103, 0.72146836, 0.11577059,
       0.06256206, 0.0601974 , 0.06256206, 0.06256206, 0.00710103,
       0.06256206, 0.05308293, 0.70687712, 0.00710103, 0.05308293,
       0.05308293, 0.0601974 , 0.70687712, 0.05308293, 0.95874908,
       0.72146836, 0.00710103, 0.11314278, 0.72146836, 0.0601974 ,
       0.0601974 , 0.0601974 , 0.70687712, 0.70687712, 0.72146836,
       0.96192433, 0.11577059, 0.72146836, 0.00710103, 0.0601974 ,
       0.05308293, 0.0601974 , 0.0601974 , 0.70687712, 0.00710103,
       0.06256206, 0.70687712, 0.70687712, 0.00710103, 0.0601974 ,
       0.00710103, 0.72146836, 0.72335222, 0.70687712, 0.70687712,
       0.72335222, 0.0601974 , 0.70687712, 0.95874908, 0.72335222,
       0.72146836, 0.96192433, 0.0601974 , 0.11314278, 0.70687712,
       0.72335222, 0.05308293, 0.00710103, 0.00710103, 0.05308293,
       0.05308293, 0.0601974 , 0.0601974 , 0.11577059, 0.11577

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
605,KEN,Rugby Sevens,False,0,0,0,6,7,0
950,COL,Rugby Sevens,False,0,0,0,2,7,0
951,COL,Rugby Sevens,False,0,0,0,2,7,0
1311,KEN,Rugby Sevens,False,0,0,0,6,7,0
1555,ZAF,Rugby Sevens,True,0,0,0,3,6,0
...,...,...,...,...,...,...,...,...,...
197026,NZL,Rugby Sevens,True,0,1,0,1,8,0
197656,USA,Rugby Sevens,False,0,2,1,1,8,5
198269,JPN,Rugby Sevens,False,0,1,0,0,8,1
198376,JPN,Rugby Sevens,False,0,1,0,0,8,1


In [201]:
Rugby_df['Predictions'] = y_pred.tolist()
Rugby_final = Rugby_df.drop_duplicates(subset=['NOC'], keep='last')
Rugby_final = Rugby_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3353199450.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Rugby_final = Rugby_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [202]:
P = pd.merge(Softball_final, Rugby_final, how='outer', on=['NOC', 'NOC'])
P.rename(columns = {'Predictions_x':'Softball', 'Predictions_y':'Rugby Sevens'}, inplace = True)
P = P.drop(["Sport_x","Sport_y"], 1)
P

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3810029682.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  P = P.drop(["Sport_x","Sport_y"], 1)


,NOC,Softball,Rugby Sevens
0,GRC,0.019436,NaN
1,VEN,0.022620,NaN
2,PRI,0.000148,NaN
3,ITA,0.019436,NaN
4,NLD,0.080752,NaN
5,NZL,0.022620,0.706877
6,USA,0.999678,0.053083
7,AUS,0.971448,0.706877
8,JPN,0.957386,0.060201
9,CAN,0.009417,0.958749


Trampolining

In [203]:
Trampolining_df = data_df[data_df['Sport'] == "Trampolining"]
Trampolining_df = Trampolining_df.dropna()

In [204]:
Trampolining_dummies = pd.get_dummies(Trampolining_df)
Trampolining_dummies.head(3)

,Medal,Host,NOC_AUS,NOC_BLR,NOC_BRA,NOC_CAN,NOC_CHE,NOC_CHN,NOC_CZE,NOC_DEU,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
803,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1099,False,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
2029,False,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [205]:
x_train = Trampolining_dummies.drop(columns=['Medal'])
y_train = Trampolining_dummies['Medal']

x_test = Trampolining_dummies.drop(columns=['Medal'])
y_test = Trampolining_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.8552631578947368

In [206]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Trampolining_df)

array([0.22930634, 0.48289323, 0.11769911, 0.43064804, 0.42806883,
       0.24945491, 0.11769911, 0.43064804, 0.24945491, 0.69897815,
       0.69897815, 0.84276467, 0.24945491, 0.17380828, 0.17380828,
       0.4678212 , 0.69704035, 0.3237568 , 0.84276467, 0.84276467,
       0.69897815, 0.69897815, 0.29491311, 0.48289323, 0.48289323,
       0.288032  , 0.288032  , 0.93995686, 0.93613122, 0.86560101,
       0.11769911, 0.26198519, 0.24945491, 0.11769911, 0.3237568 ,
       0.30878735, 0.17380828, 0.15399761, 0.22930634, 0.22930634,
       0.86560101, 0.29491311, 0.1534041 , 0.11769911, 0.32103166,
       0.32103166, 0.32103166, 0.3237568 , 0.3237568 , 0.93995686,
       0.93613122, 0.86560101, 0.15399761, 0.15399761, 0.80329576,
       0.93995686, 0.93613122, 0.07309489, 0.15399761, 0.12586454,
       0.12586454, 0.12586454, 0.24945491, 0.69704035, 0.69704035,
       0.3237568 , 0.3237568 , 0.3237568 , 0.3237568 , 0.43064804,
       0.43064804, 0.43064804, 0.43064804, 0.43064804, 0.32103

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
803,PRT,Trampolining,False,0,0,0,3,8,0
1099,DEU,Trampolining,False,0,1,0,2,8,1
2029,USA,Trampolining,False,0,2,1,1,8,5
3963,KAZ,Trampolining,False,0,0,0,4,7,0
5722,BRA,Trampolining,False,1,0,1,1,7,0
...,...,...,...,...,...,...,...,...,...
192188,AUS,Trampolining,True,1,1,0,1,8,0
195202,CHE,Trampolining,False,0,1,0,2,8,0
195374,AUS,Trampolining,False,0,2,0,1,8,0
197277,GBR,Trampolining,False,0,2,0,1,8,1


In [207]:
Trampolining_df['Predictions'] = y_pred.tolist()
Trampolining_final = Trampolining_df.drop_duplicates(subset=['NOC'], keep='last')
Trampolining_final = Trampolining_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/458389408.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Trampolining_final = Trampolining_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


Golf

In [208]:
Golf_df = data_df[data_df['Sport'] == "Golf"]
Golf_df = Golf_df.dropna()

In [209]:
Golf_dummies = pd.get_dummies(Golf_df)
Golf_dummies.head(3)

,Medal,Host,NOC_ARG,NOC_AUS,NOC_AUT,NOC_BEL,NOC_BGD,NOC_BRA,NOC_CAN,NOC_CHE,...,Life_Expect_Bin_8,Global_GDP_Perc_Bin_0,Global_GDP_Perc_Bin_1,Global_GDP_Perc_Bin_2,Global_GDP_Perc_Bin_3,Global_GDP_Perc_Bin_4,Global_GDP_Perc_Bin_5,Global_GDP_Perc_Bin_6,Global_GDP_Perc_Bin_7,Global_GDP_Perc_Bin_8
1755,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
5152,False,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
6701,False,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [210]:
x_train = Golf_dummies.drop(columns=['Medal'])
y_train = Golf_dummies['Medal']

x_test = Golf_dummies.drop(columns=['Medal'])
y_test = Golf_dummies['Medal']

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

0.9491525423728814

In [211]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)

clf.fit(x_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(x_test)
y_pred = y_pred[:,1]

display(y_pred,Golf_df)

array([0.07933427, 0.70852232, 0.0338644 , 0.14227002, 0.07933427,
       0.07933427, 0.19737095, 0.19737095, 0.19737095, 0.17445496,
       0.14227002, 0.0338644 , 0.70852232, 0.19737095, 0.07933427,
       0.32126957, 0.19737095, 0.65331279, 0.74050963, 0.81139127,
       0.07933427, 0.17445496, 0.19737095, 0.07933427, 0.07933427,
       0.0338644 , 0.0338644 , 0.07933427, 0.0338644 , 0.07933427,
       0.19737095, 0.07933427, 0.19737095, 0.77392438, 0.19737095,
       0.17445496, 0.19737095, 0.07933427, 0.0338644 , 0.0338644 ,
       0.19737095, 0.17445496, 0.17445496, 0.70852232, 0.19737095,
       0.81139127, 0.19737095, 0.0338644 , 0.74050963, 0.65331279,
       0.14227002, 0.19737095, 0.81139127, 0.07933427, 0.07933427,
       0.74050963, 0.65331279, 0.07933427, 0.65331279, 0.65331279,
       0.77390985, 0.77390985, 0.19737095, 0.19737095, 0.32126957,
       0.65331279, 0.07933427, 0.19737095, 0.07933427, 0.07933427,
       0.17445496, 0.77392438, 0.07933427, 0.07933427, 0.19737

,NOC,Sport,Medal,Host,GDP_Per_Cap,Global_Pop_Bin,Rural_Pop_Bin,Life_Expect_Bin,Global_GDP_Perc_Bin
1755,CHL,Golf,False,0,0,0,0,8,0
5152,KOR,Golf,False,0,1,0,1,8,0
6701,THA,Golf,False,0,0,0,4,7,0
7986,IND,Golf,False,0,0,7,6,7,0
13296,ISR,Golf,False,0,1,0,0,8,0
...,...,...,...,...,...,...,...,...,...
195547,GBR,Golf,False,0,1,0,1,8,0
196704,AUT,Golf,False,0,2,0,3,8,0
197358,CHN,Golf,False,0,0,7,4,8,3
198605,KOR,Golf,False,0,1,0,1,8,0


In [212]:
Golf_df['Predictions'] = y_pred.tolist()
Golf_final = Golf_df.drop_duplicates(subset=['NOC'], keep='last')
Golf_final = Golf_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/3088466276.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Golf_final = Golf_final.drop(["Host","GDP_Per_Cap", "Global_Pop_Bin", "Rural_Pop_Bin", "Life_Expect_Bin", "Global_GDP_Perc_Bin", "Medal"], 1)


In [213]:
Q = pd.merge(Trampolining_final, Golf_final, how='outer', on=['NOC', 'NOC'])
Q.rename(columns = {'Predictions_x':'Trampoling', 'Predictions_y':'Golf'}, inplace = True)
Q = Q.drop(["Sport_x","Sport_y"], 1)
Q

C:\Users\14043\AppData\Local\Temp/ipykernel_1700/762997901.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Q = Q.drop(["Sport_x","Sport_y"], 1)


,NOC,Trampoling,Golf
0,KAZ,0.430648,NaN
1,BRA,0.428069,0.321270
2,DZA,0.430648,NaN
3,ITA,0.173808,0.079334
4,DNK,0.125865,0.197371
5,UZB,0.430648,NaN
6,GEO,0.321032,NaN
7,CHN,0.803296,0.653313
8,GRC,0.173808,NaN
9,FRA,0.249455,0.197371


In [214]:
First = pd.merge(A, B, how='outer', on=['NOC', 'NOC'])
First

,NOC,Athletics,Swimming,Gymnastics,Cycling
0,OMN,0.056710,0.035608,NaN,0.082364
1,PLW,0.142405,0.035608,NaN,NaN
2,SOM,0.039474,NaN,NaN,NaN
3,AGO,0.039474,0.006516,NaN,NaN
4,BDI,0.394120,0.007109,NaN,NaN
...,...,...,...,...,...
182,DEU,0.464116,0.635384,0.302244,0.766613
183,NLD,0.186889,0.642624,0.258100,0.579867
184,USA,0.858066,0.964731,0.631431,0.008193
185,KAZ,0.268219,0.073415,0.017179,0.389322


In [215]:
Second = pd.merge(C, D, how='outer', on=['NOC', 'NOC'])
Second

,NOC,Rowing,Shooting,Fencing,Canoeing
0,KWT,0.192253,0.534172,0.031138,NaN
1,TGO,0.008707,NaN,NaN,0.689700
2,HND,0.091612,0.190861,0.043265,NaN
3,TTO,0.187417,0.145890,NaN,NaN
4,CMR,0.103862,NaN,NaN,NaN
...,...,...,...,...,...
136,ATG,NaN,NaN,NaN,0.030549
137,SYC,NaN,NaN,NaN,0.106965
138,MOZ,NaN,NaN,NaN,0.023624
139,GUM,NaN,NaN,NaN,0.310492


In [216]:
Third = pd.merge(E, F, how='outer', on=['NOC', 'NOC'])
Third

,NOC,Wrestling,Football,Boxing,Equestrianism
0,YEM,0.024006,NaN,NaN,NaN
1,CHL,0.196641,0.892639,0.094670,0.008916
2,BHR,0.298727,NaN,NaN,NaN
3,LBN,0.165723,NaN,0.274829,NaN
4,CIV,0.050638,0.000010,0.084550,NaN
...,...,...,...,...,...
158,ETH,NaN,NaN,0.133535,NaN
159,TON,NaN,NaN,0.257779,NaN
160,JAM,NaN,NaN,0.061544,0.112318
161,PNG,NaN,NaN,0.096735,NaN


In [217]:
Fourth = pd.merge(G, H, how='outer', on=['NOC', 'NOC'])
Fourth

,NOC,Judo,Basketball,Handball,Volleyball
0,GUM,0.133986,NaN,NaN,NaN
1,TGO,0.068382,NaN,NaN,NaN
2,NIC,0.180143,NaN,NaN,NaN
3,TTO,0.180143,NaN,NaN,NaN
4,BEN,0.068382,NaN,NaN,NaN
...,...,...,...,...,...
152,HUN,0.430885,NaN,0.574258,NaN
153,CHL,0.174665,NaN,NaN,NaN
154,CAN,0.194549,0.002093,0.014495,0.018650
155,GEO,0.579907,NaN,NaN,NaN


In [218]:
Fifth = pd.merge(I, J, how='outer', on=['NOC', 'NOC'])
Fifth

,NOC,Weightlifting,Water Polo,Archery,Diving
0,SAU,0.199698,NaN,0.011219,NaN
1,GHA,0.055781,NaN,NaN,NaN
2,ZWE,0.018348,NaN,0.026757,0.025323
3,SLE,0.060120,NaN,NaN,NaN
4,LBN,0.199698,NaN,NaN,NaN
...,...,...,...,...,...
133,LAO,NaN,NaN,0.026757,NaN
134,CAF,NaN,NaN,0.050498,NaN
135,HKG,NaN,NaN,0.157136,0.033819
136,BTN,NaN,NaN,0.050498,NaN


In [219]:
Sixth = pd.merge(K, L, how='outer', on=['NOC', 'NOC'])
Sixth

,NOC,Tennis,Table Tennis,Badminton,Modern Pentathlon
0,MDA,0.168556,NaN,NaN,NaN
1,SLV,0.042128,NaN,NaN,NaN
2,LTU,0.042128,0.006639,0.001491,0.790233
3,TUR,0.117699,0.006639,0.033898,0.111344
4,BOL,0.030831,NaN,NaN,NaN
...,...,...,...,...,...
116,LKA,NaN,NaN,0.063613,NaN
117,GTM,NaN,NaN,0.042543,0.070224
118,BRN,NaN,NaN,0.016013,NaN
119,BHR,NaN,NaN,NaN,0.153452


In [220]:
Seventh = pd.merge(M, N, how='outer', on=['NOC', 'NOC'])
Seventh

,NOC,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo
0,COL,0.020026,NaN,NaN,0.391372
1,SWE,0.031136,NaN,0.136182,0.175473
2,SVK,0.020026,NaN,NaN,NaN
3,FIN,0.031136,NaN,0.076925,0.255469
4,HUN,0.017297,NaN,0.072178,0.233055
...,...,...,...,...,...
105,THA,NaN,NaN,NaN,0.667139
106,ARM,NaN,NaN,NaN,0.233055
107,GTM,NaN,NaN,NaN,0.293212
108,MAR,NaN,NaN,NaN,0.112251


In [221]:
Eighth = pd.merge(O, P, how='outer', on=['NOC', 'NOC'])
Eighth 

,NOC,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,TUN,0.027404,NaN,NaN,NaN
1,CRI,0.072555,0.023700,NaN,NaN
2,AGO,0.002971,NaN,NaN,NaN
3,CHL,0.083075,0.033364,NaN,NaN
4,NZL,0.040324,0.069118,0.022620,0.706877
5,EGY,0.002971,NaN,NaN,NaN
6,GBR,0.062799,0.857441,NaN,0.721468
7,PRI,0.083075,NaN,0.000148,NaN
8,SWE,0.044588,0.788051,NaN,NaN
9,POL,0.027404,0.023700,NaN,NaN


In [222]:
A = pd.merge(First, Second, how='outer', on=['NOC', 'NOC'])
A 

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing
0,OMN,0.056710,0.035608,NaN,0.082364,NaN,0.086522,NaN,NaN
1,PLW,0.142405,0.035608,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039474,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039474,0.006516,NaN,NaN,0.008707,0.007468,NaN,0.160373
4,BDI,0.394120,0.007109,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
183,NLD,0.186889,0.642624,0.258100,0.579867,0.589525,0.192910,0.256265,0.336485
184,USA,0.858066,0.964731,0.631431,0.008193,0.404299,0.775181,0.214388,0.330411
185,KAZ,0.268219,0.073415,0.017179,0.389322,0.091612,0.412283,0.008770,0.027573
186,RUS,0.735717,0.499291,0.786953,0.543098,0.382792,0.875006,0.838182,0.586206


In [223]:
B = pd.merge(Third, Fourth, how = 'outer', on=['NOC','NOC'])
B

,NOC,Wrestling,Football,Boxing,Equestrianism,Judo,Basketball,Handball,Volleyball
0,YEM,0.024006,NaN,NaN,NaN,0.002548,NaN,NaN,NaN
1,CHL,0.196641,0.892639,0.094670,0.008916,0.174665,NaN,NaN,NaN
2,BHR,0.298727,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LBN,0.165723,NaN,0.274829,NaN,0.172860,NaN,NaN,NaN
4,CIV,0.050638,0.000010,0.084550,NaN,0.035572,NaN,0.009617,NaN
...,...,...,...,...,...,...,...,...,...
169,ISL,NaN,NaN,NaN,NaN,0.236401,NaN,0.478225,NaN
170,RWA,NaN,NaN,NaN,NaN,0.048661,NaN,NaN,NaN
171,ABW,NaN,NaN,NaN,NaN,0.180143,NaN,NaN,NaN
172,MMR,NaN,NaN,NaN,NaN,0.079584,NaN,NaN,NaN


In [224]:
C = pd.merge(Fifth, Sixth, how = 'outer', on=['NOC','NOC'])
C

,NOC,Weightlifting,Water Polo,Archery,Diving,Tennis,Table Tennis,Badminton,Modern Pentathlon
0,SAU,0.199698,NaN,0.011219,NaN,NaN,0.006639,NaN,NaN
1,GHA,0.055781,NaN,NaN,NaN,NaN,0.007042,NaN,NaN
2,ZWE,0.018348,NaN,0.026757,0.025323,0.030289,NaN,NaN,NaN
3,SLE,0.060120,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LBN,0.199698,NaN,NaN,NaN,NaN,0.002925,NaN,NaN
...,...,...,...,...,...,...,...,...,...
148,ZMB,NaN,NaN,NaN,NaN,NaN,NaN,0.055189,NaN
149,MDV,NaN,NaN,NaN,NaN,NaN,NaN,0.055189,NaN
150,SUR,NaN,NaN,NaN,NaN,NaN,NaN,0.001491,NaN
151,BRN,NaN,NaN,NaN,NaN,NaN,NaN,0.016013,NaN


In [225]:
D = pd.merge(Seventh, Eighth, how = 'outer', on=['NOC','NOC'])
D

,NOC,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,COL,0.020026,NaN,NaN,0.391372,NaN,0.037676,NaN,0.113143
1,SWE,0.031136,NaN,0.136182,0.175473,0.044588,0.788051,NaN,NaN
2,SVK,0.020026,NaN,NaN,NaN,NaN,0.009910,NaN,NaN
3,FIN,0.031136,NaN,0.076925,0.255469,NaN,NaN,NaN,NaN
4,HUN,0.017297,NaN,0.072178,0.233055,NaN,0.037676,NaN,NaN
...,...,...,...,...,...,...,...,...,...
113,ZWE,NaN,NaN,NaN,NaN,NaN,0.037676,NaN,NaN
114,LUX,NaN,NaN,NaN,NaN,NaN,0.468538,NaN,NaN
115,IRL,NaN,NaN,NaN,NaN,NaN,0.376789,NaN,NaN
116,BRB,NaN,NaN,NaN,NaN,NaN,0.037676,NaN,NaN


In [227]:
First = pd.merge(A, B, how = 'outer', on=['NOC','NOC'])
First

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,Football,Boxing,Equestrianism,Judo,Basketball,Handball,Volleyball
0,OMN,0.056710,0.035608,NaN,0.082364,NaN,0.086522,NaN,NaN,NaN,NaN,0.270712,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142405,0.035608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039474,0.006516,NaN,NaN,0.008707,0.007468,NaN,0.160373,NaN,NaN,0.063665,NaN,0.060366,0.012225,0.022974,NaN
4,BDI,0.394120,0.007109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061310,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186889,0.642624,0.258100,0.579867,0.589525,0.192910,0.256265,0.336485,0.334028,0.103058,0.551324,0.484646,0.643769,NaN,0.039963,0.426139
184,USA,0.858066,0.964731,0.631431,0.008193,0.404299,0.775181,0.214388,0.330411,0.767259,0.018143,0.793294,0.819421,0.538368,0.986471,0.000240,0.519972
185,KAZ,0.268219,0.073415,0.017179,0.389322,0.091612,0.412283,0.008770,0.027573,0.712701,NaN,0.800936,NaN,0.297569,NaN,0.027414,0.124548
186,RUS,0.735717,0.499291,0.786953,0.543098,0.382792,0.875006,0.838182,0.586206,0.924468,NaN,0.843051,0.104262,0.694135,0.880470,0.804584,0.738512


In [228]:
Second = pd.merge(C, D, how = 'outer', on=['NOC','NOC'])
Second

,NOC,Weightlifting,Water Polo,Archery,Diving,Tennis,Table Tennis,Badminton,Modern Pentathlon,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,SAU,0.199698,NaN,0.011219,NaN,NaN,0.006639,NaN,NaN,NaN,NaN,NaN,0.406888,NaN,NaN,NaN,NaN
1,GHA,0.055781,NaN,NaN,NaN,NaN,0.007042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZWE,0.018348,NaN,0.026757,0.025323,0.030289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037676,NaN,NaN
3,SLE,0.060120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LBN,0.199698,NaN,NaN,NaN,NaN,0.002925,NaN,NaN,NaN,NaN,NaN,0.272043,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,GAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.406888,NaN,NaN,NaN,NaN
160,KHM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.233055,NaN,NaN,NaN,NaN
161,GRD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.157718,NaN,NaN,NaN,NaN
162,LSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.233055,NaN,NaN,NaN,NaN


In [229]:
A = pd.merge(First, Second, how = 'outer', on=['NOC','NOC'])
A

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,...,Badminton,Modern Pentathlon,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens
0,OMN,0.056710,0.035608,NaN,0.082364,NaN,0.086522,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142405,0.035608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039474,0.006516,NaN,NaN,0.008707,0.007468,NaN,0.160373,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.002971,NaN,NaN,NaN
4,BDI,0.394120,0.007109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186889,0.642624,0.258100,0.579867,0.589525,0.192910,0.256265,0.336485,0.334028,...,0.226756,0.166424,0.048325,0.025424,NaN,0.263080,0.368930,0.088852,0.080752,NaN
184,USA,0.858066,0.964731,0.631431,0.008193,0.404299,0.775181,0.214388,0.330411,0.767259,...,0.038650,0.472313,0.772863,0.968662,0.035280,0.765530,0.767346,0.530353,0.999678,0.053083
185,KAZ,0.268219,0.073415,0.017179,0.389322,0.091612,0.412283,0.008770,0.027573,0.712701,...,NaN,0.490180,0.020026,NaN,0.072178,0.293212,NaN,0.140698,NaN,NaN
186,RUS,0.735717,0.499291,0.786953,0.543098,0.382792,0.875006,0.838182,0.586206,0.924468,...,0.274320,0.849728,0.456980,NaN,0.980291,0.566208,0.072555,0.037676,NaN,NaN


In [230]:
Final = pd.merge(A, Q, how = 'outer', on=['NOC','NOC'])
Final

,NOC,Athletics,Swimming,Gymnastics,Cycling,Rowing,Shooting,Fencing,Canoeing,Wrestling,...,Sychronized Swimming,Baseball,Rhythmic Gymnastics,Taekwondo,Beach Volleyball,Triathlon,Softball,Rugby Sevens,Trampoling,Golf
0,OMN,0.056710,0.035608,NaN,0.082364,NaN,0.086522,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLW,0.142405,0.035608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOM,0.039474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGO,0.039474,0.006516,NaN,NaN,0.008707,0.007468,NaN,0.160373,NaN,...,NaN,NaN,NaN,NaN,0.002971,NaN,NaN,NaN,NaN,NaN
4,BDI,0.394120,0.007109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NLD,0.186889,0.642624,0.258100,0.579867,0.589525,0.192910,0.256265,0.336485,0.334028,...,0.048325,0.025424,NaN,0.263080,0.368930,0.088852,0.080752,NaN,0.249455,0.079334
184,USA,0.858066,0.964731,0.631431,0.008193,0.404299,0.775181,0.214388,0.330411,0.767259,...,0.772863,0.968662,0.035280,0.765530,0.767346,0.530353,0.999678,0.053083,0.117699,0.740510
185,KAZ,0.268219,0.073415,0.017179,0.389322,0.091612,0.412283,0.008770,0.027573,0.712701,...,0.020026,NaN,0.072178,0.293212,NaN,0.140698,NaN,NaN,0.430648,NaN
186,RUS,0.735717,0.499291,0.786953,0.543098,0.382792,0.875006,0.838182,0.586206,0.924468,...,0.456980,NaN,0.980291,0.566208,0.072555,0.037676,NaN,NaN,0.323757,0.033864


In [235]:
Final.to_csv(r'archive\countryPredictions.csv', index = False)